In [2]:
import plotly.express as px
import pandas as pd
import plotly.express as px
import plotly.io as pio
import json 
import plotly
import numpy as np
import json 
import geopandas as gpd
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt

df_orig = pd.read_csv("../viz_data/mapc_data.csv")

df = df_orig[df_orig['price'] < 10000000]
df = df[df['yearbuilt'] > 0]


# Load the geojson file
gdf = gpd.read_file('../viz_data/2020_census_mapc.geojson')
df['geoid'] = df['ct20_id'].convert_dtypes(convert_floating=False)
gdf['geoid'] = pd.to_numeric(gdf['geoid'])
df = pd.merge(df, gdf, on='geoid', how='inner')
census_df = pd.read_csv('../viz_data/aggregated2020.csv')
census_df = census_df.rename(columns={'lat': 'latin'})
census_df['geoid'] = census_df['ct20_id'].convert_dtypes(convert_floating=False)
df = pd.merge(df, census_df, on='geoid', how='inner')
df['year'] = pd.to_datetime(df['date']).dt.year
mdf = pd.read_csv('../viz_data/MAPCPI.csv')
mdf['year'] = pd.to_datetime(mdf['DATE']).dt.year
inv_color_mapping = {
    'Small' : 'blue',
    'Medium' : 'orange',
    'Large' : 'green',
    'Institutional' : 'red'
}
df['flip_status'] = df['buy_side_flip'].map({0: 'NoFlip', 1: 'Flip'})

# Define style mapping
style_mapping = {
    # Traditional
    **{style: 'Traditional' for style in ['COLONIAL', 'CONVENTIONAL', 'CAPE COD', 'RANCH', 'OLD STYLE', 'VICTORIAN', 'ANTIQUE', 'GARRISON', 'FEDERALIST', 'SALT BOX','COTTAGE', 'MANSION', 'RAISED CAPE', 'GAMBREL', 'BI-LEVEL']},
    # Modern
    **{style: 'Modern' for style in ['CAMP/CABIN', 'MOBILE HOME','CONTEMPORARY', 'MID-RISE', 'HIGH-RISE', 'LOW-RISE', 'FREE-STANDNG', 'RAISED RANCH', 'SPLIT LEVEL', 'SPLIT ENTRY', 'TUDOR', 'LOFT', 'CUSTOMDESIGN']},
    # Multi-Unit
    **{style: 'Multi-Unit' for style in ['CONDO/APT', 'TOWNHOUSE', 'MLTI-UNT BLG', 'DECKER', 'TOWNHSE-END', 'MIXED BLDGS', 'ROW-END', 'ROW-MIDDLE', 'ROW HOUSE', 'STACK', 'THREE STORY', 'TWO STORY', 'LODGE', 'CARRIAGE HSE', 'DUPLEX', 'SEMI DETACHD']},
}

# Assign style category to each style
df['style_category'] = df['style'].map(style_mapping)

city_to_subregion = {
    'Arlington': 'ICC', 'Belmont': 'ICC', 'Boston': 'ICC', 'Brookline': 'ICC',
    'Cambridge': 'ICC', 'Chelsea': 'ICC', 'Everett': 'ICC', 'Lynn': 'ICC',
    'Malden': 'ICC', 'Medford': 'ICC', 'Melrose': 'ICC', 'Milton': 'ICC',
    'Needham': 'ICC', 'Newton': 'ICC', 'Quincy': 'ICC', 'Revere': 'ICC',
    'Saugus': 'ICC', 'Somerville': 'ICC', 'Waltham': 'ICC', 'Watertown': 'ICC',
    'Winthrop': 'ICC',
    
    'Ashland': 'MWRC', 'Framingham': 'MWRC', 'Holliston': 'MWRC',
    'Marlborough': 'MWRC', 'Natick': 'MWRC', 'Sherborn': 'MWRC',
    'Southborough': 'MWRC', 'Wayland': 'MWRC', 'Wellesley': 'MWRC',
    'Weston': 'MWRC',
    
    'Acton': 'MAGIC', 'Bedford': 'MAGIC', 'Bolton': 'MAGIC',
    'Boxborough': 'MAGIC', 'Carlisle': 'MAGIC', 'Concord': 'MAGIC',
    'Hudson': 'MAGIC', 'Lexington': 'MAGIC', 'Lincoln': 'MAGIC',
    'Littleton': 'MAGIC', 'Maynard': 'MAGIC', 'Stow': 'MAGIC',
    'Sudbury': 'MAGIC',
    
    'Beverly': 'NSTF', 'Danvers': 'NSTF', 'Essex': 'NSTF',
    'Gloucester': 'NSTF', 'Ipswich': 'NSTF', 'Hamilton': 'NSTF',
    'Manchester': 'NSTF', 'Marblehead': 'NSTF', 'Middleton': 'NSTF',
    'Nahant': 'NSTF', 'Peabody': 'NSTF', 'Rockport': 'NSTF',
    'Salem': 'NSTF', 'Swampscott': 'NSTF', 'Topsfield': 'NSTF', 'Wenham' : 'NSTF',
    
    'Burlington': 'NSPC', 'Lynnfield': 'NSPC', 'North Reading': 'NSPC',
    'Reading': 'NSPC', 'Stoneham': 'NSPC', 'Wakefield': 'NSPC',
    'Wilmington': 'NSPC', 'Winchester': 'NSPC', 'Woburn': 'NSPC',
    
    'Braintree': 'SSC', 'Cohasset': 'SSC', 'Duxbury': 'SSC',
    'Hanover': 'SSC', 'Hingham': 'SSC', 'Holbrook': 'SSC',
    'Hull': 'SSC', 'Marshfield': 'SSC', 'Norwell': 'SSC',
    'Pembroke': 'SSC', 'Rockland': 'SSC', 'Scituate': 'SSC',
    'Weymouth': 'SSC',
    
    'Bellingham': 'SWAP', 'Dover': 'SWAP', 'Franklin': 'SWAP',
    'Hopkinton': 'SWAP', 'Medway': 'SWAP', 'Milford': 'SWAP',
    'Millis': 'SWAP', 'Norfolk': 'SWAP', 'Sherborn': 'SWAP',
    'Wrentham': 'SWAP',
    
    'Canton': 'TRIC', 'Dedham': 'TRIC', 'Dover': 'TRIC',
    'Foxborough': 'TRIC', 'Medfield': 'TRIC', 'Milton': 'TRIC',
    'Needham': 'TRIC', 'Norwood': 'TRIC', 'Randolph': 'TRIC',
    'Sharon': 'TRIC', 'Stoughton': 'TRIC', 'Walpole': 'TRIC',
    'Westwood': 'TRIC'
}

# Assuming your DataFrame is called df and the city column is called 'city'
df['subregion'] = df['city'].map(city_to_subregion)

def col_to_fillcol(col, alpha):
    rgba = mcolors.to_rgba(col)
    rgba = tuple([int(255*x) for x in rgba])[:-1] + (alpha,)
    return 'rgba' + str(rgba)


In [18]:

# INV CHOOSE plot for each inv type

import pandas as pd
import plotly.express as px

for inv_type in ['Small', 'Medium', 'Large', 'Institutional']:
    print(inv_type)
    try:
        inv_col = inv_color_mapping[inv_type]  # Assuming inv_color_mapping is defined somewhere with appropriate colors

        # Assume df is already loaded into the workspace
        # Convert 'date' to datetime and extract year

        # Sort the unique years and get the last five years
        sorted_years = sorted(df['year'].unique())
        last_five_years = sorted_years[-5:]

        # Example for one investor type, e.g., 'Institutional'
        df_small = df[(df['investor_type_sale'] == inv_type) & (df['year'].isin(last_five_years))]

        # Sort data by year
        df_small = df_small.sort_values('year')

        # Calculate the 2.5th and 97.5th percentiles of the profit percentage
        lower_bound, upper_bound = df_small['price_diff_pch'].quantile([0.025, 0.95])

        # Filter the data to the middle 95%
        df_small_filtered = df_small[(df_small['price_diff_pch'] >= lower_bound) & 
                                    (df_small['price_diff_pch'] <= upper_bound)]

        # Create a violin plot for each year, animated by year
        fig = px.violin(df_small_filtered, y='price_diff_pch', animation_frame='year', 
                        points="all", box=True, labels={"price_diff_pch": "Profit Percentage"})
        fig.update_traces(marker=dict(color=inv_col), line=dict(color=inv_col))

        # Improve layout and aesthetics
        fig.update_layout(
            yaxis_title="Profit Percentage",
            xaxis_title="Year",
            xaxis_visible=False,  # Hide the x-axis ticks and labels
            plot_bgcolor='white'  # Set background color to white
        )

        # Add a line at Profit = 0 for reference
        fig.add_shape(
            type="line",
            x0=0.05, x1=0.9, y0=0, y1=0,  # Adjust x0 and x1 for proper positioning if necessary
            line=dict(
                color="Black",
                width=1
            ),
            xref="paper",  # Use 'paper' to span the line across the full axis
            yref="y"
        )

        # Set the color of the violin plot and jitter points to the selected color for all frames
        for frame in fig.frames:
            frame.data[0].update(marker=dict(color=inv_col), line=dict(color=inv_col))

        # Center the title of the plot
        fig.update_layout(
            title={
                'text': f"Profit % for <b>{inv_type}</b> Investors (last 5 years)",
                'y':0.9,
                'x':0.5,
                'xanchor': 'center',
                'yanchor': 'top'
            },
            title_font_size=16
        )


        plot_json = json.dumps(fig, cls=plotly.utils.PlotlyJSONEncoder)
        with open(f'investor-profiles/public/inv_pre/{inv_type}_profit.json', 'w') as file:
            file.write(plot_json)
    except Exception as error:
        # handle the exception
        print("An exception occurred:", type(error).__name__, "–", error)        
        print("###################################### FAILED TO GENERATE ######################################")

Small
Medium
Large
Institutional


In [9]:
# INV choose --> bipartite graph 

import pandas as pd
import plotly.graph_objects as go
import numpy as np  # Import numpy if not already imported

# Assuming df is already loaded into the workspace
# Assuming inv_color_mapping is defined somewhere in your code, like:
# inv_color_mapping = {'Small': 'red', 'Medium': 'green', 'Large': 'blue', 'Institutional': 'purple'}

# Filter transactions to include only those involving investor types as both buyers and sellers
df_filtered = df[
    (df['investor_type_sale'].isin(['Small', 'Medium', 'Large', 'Institutional'])) &
    (df['investor_type_purchase'].isin(['Small', 'Medium', 'Large', 'Institutional']))
]

# Calculate the volume of transactions between each type of investor
transaction_volume = df_filtered.groupby(['investor_type_sale', 'investor_type_purchase']).size().reset_index(name='volume')

# Create the plot
fig = go.Figure()

# Positions for the investor types on the plot
positions = {'Small': 1, 'Medium': 2, 'Large': 3, 'Institutional': 4}

# Define a color for the lines that is aesthetically pleasing
line_color = 'black'  # Example: a shade of green

# Add curved lines for each transaction
for index, row in transaction_volume.iterrows():
    seller_pos = positions[row['investor_type_sale']]
    buyer_pos = positions[row['investor_type_purchase']]
    volume = row['volume']

    # Add a smooth curved line representing the transaction
    fig.add_trace(go.Scatter(
        x=[seller_pos, (seller_pos + buyer_pos) / 2, buyer_pos],
        y=[1, np.random.choice([0.3, 0.4, 0.5, 0.6, 0.7]), 0],  # Randomly choosing midpoint y to make the curve smoother
        mode='lines',
        line=dict(width=volume / max(transaction_volume['volume']) * 10, color=line_color),
        hoverinfo='text',
        text=f"{row['investor_type_sale']} to {row['investor_type_purchase']}: {volume}",
        textposition="top center"
    ))

# Add a single label for buyers and sellers
fig.add_trace(go.Scatter(
    x=[sum(positions.values()) / len(positions), sum(positions.values()) / len(positions)],
    y=[-0.2, 1.2],
    mode='text',
    text=["Buyers", "Sellers"],
    textposition=["bottom center", "top center"]
))

# Add nodes for sellers and buyers with text labels
for position, name in positions.items():
    # Seller nodes
    fig.add_trace(go.Scatter(
        x=[name],
        y=[1],
        text=[position],  # Adding text for each node
        mode='markers+text',
        textposition='top center',
        marker=dict(size=15, color=inv_color_mapping[position]),
        hoverinfo='none'
    ))
    
    # Buyer nodes
    fig.add_trace(go.Scatter(
        x=[name],
        y=[0],
        text=[position],  # Adding text for each node
        mode='markers+text',
        textposition='bottom center',
        marker=dict(size=15, color=inv_color_mapping[position]),
        hoverinfo='none'
    ))

# Update plot layout for better aesthetic
fig.update_layout(
    title="Transaction Volume Between Different Investor Types",
    xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
    yaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
    plot_bgcolor='white',  # Set background color to white
    showlegend=False,
)

fig.update_layout(
    title={
        'text': "Transaction Volume Between Different Investor Types",
        'y': 0.9,
        'x': 0.5,
        'xanchor': 'center',
        'yanchor': 'top'
    }
)

# Show plot
# fig.show()

plot_json = json.dumps(fig, cls=plotly.utils.PlotlyJSONEncoder)
with open(f'investor-profiles/public/inv_pre/inv_trans_volume.json', 'w') as file:
    file.write(plot_json)
   

In [4]:
# INV POST CHOOSE - first plot

import pandas as pd

# ACTIVITY HAS BOTH INV SALE AND PURCHASE

# Calculate the number of selling activities per year for each investor type
activity_sale = df.groupby(['year', 'investor_type_sale']).size().reset_index(name='activity')
activity_sale.rename(columns={'investor_type_sale': 'investor_type'}, inplace=True)

# Calculate the number of buying activities per year for each investor type
activity_purchase = df.groupby(['year', 'investor_type_purchase']).size().reset_index(name='activity')
activity_purchase.rename(columns={'investor_type_purchase': 'investor_type'}, inplace=True)

# Combine the sale and purchase data
activity = pd.concat([activity_sale, activity_purchase])

# Aggregate the combined data to sum activities of buying and selling per year per investor type
activity = activity.groupby(['year', 'investor_type']).activity.sum().reset_index()


import plotly.graph_objects as go

fig = go.Figure()

# Add a trace for each investor type
for investor in ['Small', 'Medium', 'Large', 'Institutional']:
    filtered_data = activity[activity['investor_type'] == investor]
    fig.add_trace(go.Scatter(
        x=filtered_data['year'],
        y=filtered_data['activity'],
        mode='lines',
        stackgroup='one',
        name=investor,
        line=dict(color=inv_color_mapping[investor]),
        fill='tonexty'  # Fill to the next y
    ))

# Add vertical line and annotation for 2008 housing crisis
fig.add_vline(x=2008, line=dict(color="black", width=2, dash="dot"),
              annotation_text="2008 Housing Crisis", annotation_position="top right")


# Update the layout
fig.update_layout(
    title={
        'text': "Investor Activity Frequency in Greater Boston by Year",
        'y': 0.9,
        'x': 0.5,
        'xanchor': 'center',
        'yanchor': 'top'
    },
    xaxis_title="Year",
    yaxis_title="Investor Activity Frequency",
    plot_bgcolor='white',
    showlegend=True
)

fig.show()


plot_json = json.dumps(fig, cls=plotly.utils.PlotlyJSONEncoder)
with open(f'investor-profiles/public/inv_post/inv_activity_frequency.json', 'w') as file:
    file.write(plot_json)
  

In [43]:
# INV CHOOSE - tree maps
import pandas as pd
import plotly.graph_objects as go

# Filter to include only relevant investor types
df = df[df['investor_type_purchase'].isin(['Small', 'Medium', 'Large', 'Institutional'])]

# Total investor activities across the dataset
total_activities = df['investor_type_purchase'].value_counts(normalize=True)

# Calculate price percentiles to segment data
percentiles = df['price'].quantile([0.2, 0.8])

# Define segments based on price percentiles
segments = {
    'Cheapest 20%': df[df['price'] <= percentiles[0.2]],
    'Middle 60%': df[(df['price'] > percentiles[0.2]) & (df['price'] <= percentiles[0.8])],
    'Most Expensive 20%': df[df['price'] > percentiles[0.8]]
}

# Calculate normalized activity per segment and adjust based on total activities
normalized_segments = {}
for name, segment in segments.items():
    segment_counts = segment['investor_type_sale'].value_counts()
    normalized = segment_counts / total_activities  # Normalize by global investor activity proportions
    normalized_segments[name] = normalized / normalized.sum()  # Further normalize by total per segment


# Assuming inv_color_mapping is already defined

def create_treemap(activity_data, title):
    labels = activity_data.index.tolist()
    values = activity_data.values
    colors = [inv_color_mapping[label] for label in labels if label in inv_color_mapping]
    
    fig = go.Figure(go.Treemap(
        labels=labels,
        parents=[""] * len(labels),
        values=values,
        marker_colors=colors,
        textinfo="label+percent root"
    ))

    fig.update_layout(
        margin=dict(t=50, l=25, r=25, b=25),
        title={
            'text': title,
            'y': 0.9,
            'x': 0.5,
            'xanchor': 'center',
            'yanchor': 'top'
        }
    )
    return fig


# Create and display the treemaps for each segment
figs = []
titles = ["Normalized <b>Cheapest 20%</b> Purchase Frequency", 
          "Normalized <b>Middle 60%</b> Purchase Frequency", 
          "Normalized <b>Most Expensive 20%</b> Purchase Frequency"
          ]
for title, data in zip(titles, normalized_segments.values()):
    figs.append(create_treemap(data, title))

for fig in figs:
    fig.show()

plot_json = json.dumps(figs[0], cls=plotly.utils.PlotlyJSONEncoder)
with open(f'investor-profiles/public/inv_post/cheapest_20.json', 'w') as file:
    file.write(plot_json)

plot_json = json.dumps(figs[1], cls=plotly.utils.PlotlyJSONEncoder)
with open(f'investor-profiles/public/inv_post/middle_60.json', 'w') as file:
    file.write(plot_json)

plot_json = json.dumps(figs[2], cls=plotly.utils.PlotlyJSONEncoder)
with open(f'investor-profiles/public/inv_post/expensive_20.json', 'w') as file:
    file.write(plot_json)

In [18]:
import pandas as pd
import plotly.express as px
import json
from functools import reduce
geojson_path = '../viz_data/ma_zip.json'  # This should be the path to your zip code GeoJSON file

# Load your GeoJSON file
with open(geojson_path, 'r') as f:
    geojson = json.load(f)

# Ensure zip codes in the DataFrame are properly formatted as five-character strings
df['zip'] = df['zip'].apply(lambda x: str(int(x)).zfill(5))

# Filter out transactions that do not involve the specified investor types
investor_types = ['Small', 'Medium', 'Large', 'Institutional']
df = df[df['investor_type_purchase'].isin(investor_types)]

# Calculate activity counts per zip code and investor type
activity_counts = df.groupby(['zip', 'investor_type_sale']).size().reset_index(name='activity')

# Calculate total activity per zip code
total_activity = df.groupby('zip').size().reset_index(name='total_activity')

# Merge and calculate fractions
activity_counts = activity_counts.merge(total_activity, on='zip')

common_zipcodes = reduce(lambda x, y: x.intersection(y), [
    set(activity_counts[activity_counts['investor_type_sale'] == investor_type]['zip'])
    for investor_type in inv_color_mapping.keys()
])

activity_counts = activity_counts[activity_counts['zip'].isin(common_zipcodes)]

activity_counts['fraction'] = activity_counts['activity'] / activity_counts['total_activity']


import matplotlib.colors as mcolors

def generate_color_scale(color_name):
    # Convert color name to RGBA using matplotlib
    rgba_color = mcolors.to_rgba(color_name)
    
    # Create a gradient scale from lighter to the specified color
    return [
        [0, 'rgba' + str((rgba_color[0], rgba_color[1], rgba_color[2], 0.2))],  # Lighter and more transparent
        [1, 'rgba' + str(rgba_color)]  # Full color
    ]


# Adjust the properties for geo layout correctly
for investor_type in inv_color_mapping.keys():
    print(investor_type)
    try:
        fig = px.choropleth(
            activity_counts[activity_counts['investor_type_sale'] == investor_type],
            geojson=geojson,
            locations='zip',  # DataFrame column for zip codes
            color='fraction',
            featureidkey="properties.ZCTA5CE10",  # Using the correct GeoJSON key
            scope="usa",
            color_continuous_scale=generate_color_scale(inv_color_mapping[investor_type]),
            labels={'fraction': f'Fraction of {investor_type} Activity'}
        )

        # Set the center to zip code 02141's approximate coordinates and adjust zoom
        fig.update_geos(
            fitbounds="locations",
            visible=False,
            center=dict(lat=42.3702, lon=-71.0834),  # Centering on zip code 02141
            projection_scale=20  # This scale might need adjustment for your specific visualization needs
        )

        fig.update_layout(
            title=f'Fraction of <b>{investor_type}</b> Investor Activity by Zip Code',
            margin={"r":0, "t":30, "l":0, "b":0},
            geo=dict(
                lataxis_showgrid=True, 
                lonaxis_showgrid=True,
                oceancolor="lightblue"
            )
        )


        plot_json = json.dumps(fig, cls=plotly.utils.PlotlyJSONEncoder)
        with open(f'investor-profiles/public/inv_post/{investor_type}_map.json', 'w') as file:
            file.write(plot_json)
    except Exception as error:
        # handle the exception
        print("An exception occurred:", type(error).__name__, "–", error)        
        print("###################################### FAILED TO GENERATE ######################################")


Small
Medium
Large
Institutional


In [20]:
import numpy as np
import plotly.graph_objects as go
from scipy.stats import gaussian_kde

def col_to_fillcol(col, alpha):
    rgba = mcolors.to_rgba(col)
    rgba = tuple([int(255*x) for x in rgba])[:-1] + (alpha,)
    return 'rgba' + str(rgba)

# Select the investor type for the density plot
inv_type = 'Small'
threshold = 500000

for inv_type in inv_color_mapping.keys():
    for threshold in [250000, 500000, 750000, 1000000, 1250000, 1500000]:
        print(inv_type, threshold)
        try:
            # Filter the DataFrame for the selected investor type
            df_filtered = df[df['investor_type_purchase'] == inv_type]

            # Calculate the 2.5th and 97.5th percentiles to exclude the top and bottom 2.5% of data
            lower_bound = np.percentile(df_filtered['price'], 2.5)
            upper_bound = np.percentile(df_filtered['price'], 97.5)
            df_filtered = df_filtered[(df_filtered['price'] >= lower_bound) & (df_filtered['price'] <= upper_bound)]

            # Compute KDE
            kde = gaussian_kde(df_filtered['price'])
            x_vals = np.linspace(df_filtered['price'].min(), df_filtered['price'].max(), 1000)
            y_vals = kde(x_vals)

            # Calculate the mean of the KDE
            kde_mean = np.mean(df_filtered['price'])

            # Create the figure
            fig = go.Figure()

            # Add the density line plot
            fig.add_trace(go.Scatter(x=x_vals, y=y_vals, mode='lines', name='Density', line=dict(color=inv_color_mapping[inv_type])))



            # Add shaded area below the density plot for the left side of the threshold (dark red)
            fig.add_trace(go.Scatter(x=x_vals, y=y_vals * (x_vals <= threshold), fill='tozeroy', fillcolor=col_to_fillcol(inv_color_mapping[inv_type], alpha = 0.5),
                                    line=dict(color=col_to_fillcol(inv_color_mapping[inv_type], alpha = 0)), name='Below Threshold'))
            if threshold <= df_filtered['price'].max():
                # Add shaded area below the density plot for the right side of the threshold (light red)
                fig.add_trace(go.Scatter(x=x_vals, y=y_vals * (x_vals > threshold), fill='tozeroy', fillcolor=col_to_fillcol(inv_color_mapping[inv_type], alpha = 0.1),
                                        line=dict(color=col_to_fillcol(inv_color_mapping[inv_type], alpha = 0)), name='Above Threshold'))

                # Add a black vertical line at the threshold
                fig.add_shape(type="line", x0=threshold, y0=0, x1=threshold, y1=max(y_vals),
                            line=dict(color="black", width=2), name="Threshold Line")

                # Add text next to the threshold line
                fig.add_annotation(text=f'Budget = ' + '${:,.0f}'.format(threshold), x=threshold, y=max(y_vals),
                                xshift=10, yshift=10, showarrow=False,
                                font=dict(color="black", size=12, family='Arial',))

            # Add extra xtick at the threshold location
            fig.update_layout(title=f"Density Plot of Sale Price for <b>{inv_type}</b> Investors",
                            xaxis_title='Sale Price', yaxis_title='Density',
                            showlegend=False,
                            plot_bgcolor='white')

            # Show the plot
            # fig.show()

            plot_json = json.dumps(fig, cls=plotly.utils.PlotlyJSONEncoder)
            with open(f'investor-profiles/public/budget_pre/{inv_type}_{threshold}_selector.json', 'w') as file:
                file.write(plot_json)
        except Exception as error:
            # handle the exception
            print("An exception occurred:", type(error).__name__, "–", error)        
            print("###################################### FAILED TO GENERATE ######################################")


Small 250000
Small 500000
Small 750000
Small 1000000
Small 1250000
Small 1500000
Medium 250000
Medium 500000
Medium 750000
Medium 1000000
Medium 1250000
Medium 1500000
Large 250000
Large 500000
Large 750000
Large 1000000
Large 1250000
Large 1500000
Institutional 250000
Institutional 500000
Institutional 750000
Institutional 1000000
Institutional 1250000
Institutional 1500000


In [6]:
import pandas as pd
import plotly.graph_objects as go
import matplotlib.colors as mcolors

# Define the budget and calculate segment boundaries
budget = 100000  # Defined as a variable for easy adjustments
inv_type = 'Medium'

for inv_type in inv_color_mapping.keys():
    for budget in [250000, 500000, 750000, 1000000, 1250000, 1500000]:
        print(inv_type, budget)
        try:

            segments = {
                f'${0.2 * budget/1000:.0f}K-${0.4 * budget/1000:.0f}K': (0.2 * budget, 0.4 * budget),
                f'${0.4 * budget/1000:.0f}K-${0.6 * budget/1000:.0f}K': (0.4 * budget, 0.6 * budget),
                f'${0.6 * budget/1000:.0f}K-${0.8 * budget/1000:.0f}K': (0.6 * budget, 0.8 * budget),
                f'${0.8 * budget/1000:.0f}K-${budget/1000:.0f}K': (0.8 * budget, budget),
                f'${0 * budget/1000:.0f}K-${0.2 * budget/1000:.0f}K': (0, 0.2 * budget)  # Adding this at the end to keep it circular
            }

            # Assuming df is already loaded into the workspace and inv_color_mapping is defined

            # Filter DataFrame for transactions as a Medium investor
            df_filtered = df[df['investor_type_purchase'] == inv_type]

            # Calculate the number of transactions in each segment
            segment_counts = {k: ((df_filtered['price'] > v[0]) & (df_filtered['price'] <= v[1])).sum() for k, v in segments.items()}

            # Create radar chart
            categories = list(segment_counts.keys())
            values = list(segment_counts.values())

            fig = go.Figure()

            fig.add_trace(go.Scatterpolar(
                r=values,
                theta=categories,
                fill='toself',
                fillcolor=col_to_fillcol(inv_color_mapping[inv_type], alpha = 0.5),  # Semi-transparent fill
                line=dict(color=inv_color_mapping[inv_type])
            ))

            # Customize the layout with tick values
            max_value = max(values)
            fig.update_layout(
                polar=dict(
                    radialaxis=dict(
                        visible=True,
                        tickvals=[0, 10*round(max_value/10)],
                        range=[0, max_value + 0.1 * max_value]
                    )
                ),
                title={
                    'text': f"Price Distribution Segments for <b>{inv_type}</b> Investor<br> with a budget of <b>${budget:,.0f}</b>",
                    'y':0.9,
                    'x':0.5,
                    'xanchor': 'center',
                    'yanchor': 'top'
                },
                plot_bgcolor='white'
            )

            # Show the plot
            # fig.show()

            plot_json = json.dumps(fig, cls=plotly.utils.PlotlyJSONEncoder)
            with open(f'investor-profiles/public/budget_post/{inv_type}_{budget}_radial.json', 'w') as file:
                file.write(plot_json)
        except Exception as error:
            # handle the exception
            print("An exception occurred:", type(error).__name__, "–", error)        
            print("###################################### FAILED TO GENERATE ######################################")


Small 250000
Small 500000
Small 750000
Small 1000000
Small 1250000
Small 1500000
Medium 250000
Medium 500000
Medium 750000
Medium 1000000
Medium 1250000
Medium 1500000
Large 250000
Large 500000
Large 750000
Large 1000000
Large 1250000
Large 1500000
Institutional 250000
Institutional 500000
Institutional 750000
Institutional 1000000
Institutional 1250000
Institutional 1500000


In [7]:
import pandas as pd
import plotly.graph_objects as go
import numpy as np

# NOTE I STILL NEED TO LOOP OVER BUDGETs

# Assuming df is already loaded into the workspace
budgets = [250, 500, 750, 1000, 1250, 1500]  # budgets in thousands

for inv_type in inv_color_mapping.keys():
    for budget in [250000, 500000, 750000, 1000000, 1250000, 1500000]:
        print(inv_type, budget)
        try:
            # Initialize lists to collect all bedrooms and bathrooms within the middle 95%
            all_bedrooms = []
            all_bathrooms = []

            # Calculate the global range for bedrooms and bathrooms within the middle 95% for each budget segment
            for budget_ in budgets:
                lower_bound = 0 if budget_ == 250 else (budget_ - 250) * 1000
                upper_bound = budget_ * 1000

                df_filtered = df[(df['investor_type_purchase'] == inv_type) & 
                                (df['price'] > lower_bound) & 
                                (df['price'] <= upper_bound)]

                # Append middle 95% of bedrooms and bathrooms to the lists
                all_bedrooms.append(df_filtered['bedrooms'].quantile([0.025, 0.975]).values)
                all_bathrooms.append(df_filtered['bathrooms'].quantile([0.025, 0.975]).values)

            # Flatten the lists and determine the global percentile boundaries
            global_min_bedrooms, global_max_bedrooms = np.percentile(np.hstack(all_bedrooms), [2.5, 97.5])
            global_min_bathrooms, global_max_bathrooms = np.percentile(np.hstack(all_bathrooms), [2.5, 97.5])

            # Create figure
            fig = go.Figure()

            # Process each budget segment with consistent axis ranges
            for i, budget_ in enumerate(budgets):
                lower_bound = 0 if budget_ == 250 else (budget_ - 250) * 1000
                upper_bound = budget_ * 1000

                df_filtered = df[(df['investor_type_purchase'] == inv_type) & 
                                (df['price'] > lower_bound) & 
                                (df['price'] <= upper_bound)]

                # Filter within global percentile boundaries for consistency
                df_segment = df_filtered[(df_filtered['bedrooms'].between(global_min_bedrooms, global_max_bedrooms)) &
                                        (df_filtered['bathrooms'].between(global_min_bathrooms, global_max_bathrooms))]

                # Create a 2D histogram of bedrooms vs bathrooms
                counts, xedges, yedges = np.histogram2d(df_segment['bedrooms'], df_segment['bathrooms'],
                                                        bins=[np.linspace(global_min_bedrooms, global_max_bedrooms, 10),
                                                            np.linspace(global_min_bathrooms, global_max_bathrooms, 10)])

                # Add traces for each budget with smoothed contours
                fig.add_trace(
                    go.Contour(
                        z=counts,
                        x=(xedges[:-1] + xedges[1:]) / 2,
                        y=(yedges[:-1] + yedges[1:]) / 2,
                        colorscale='Viridis',
                        contours=dict(
                            coloring='heatmap',
                            showlabels=True,  # Optionally show labels on contours
                        ),
                        visible=(i == 0),  # Only the first trace is visible initially
                        name=f'Budget ${budget_}k'
                    )
                )

            # Create sliders
            steps = []
            for i, budget_ in enumerate(budgets):
                step = dict(
                    method='update',
                    args=[{'visible': [False] * len(budgets)},
                        {'title': f'How many rooms can <b>{inv_type}</b> investors<br> afford with a Budget of <b>${budget_}k</b>'}],
                    label=f'<b>${budget_}k</b>'
                )
                step['args'][0]['visible'][i] = True  # Toggle i-th trace to "visible"
                steps.append(step)

            sliders = [dict(
                active=budgets.index(budget//1000),
                currentvalue={"prefix": "Selected Budget: "},
                steps=steps
            )]

            # Update the layout
            fig.update_layout(
                sliders=sliders,
                title=f"How many rooms can <b>{inv_type}</b> investors afford<br> with a Budget of <b>${budget//1000}k</b>",
                xaxis_title="Number of Bedrooms",
                yaxis_title="Number of Bathrooms",
                plot_bgcolor='white',
            )

            # Show plot
            # fig.show()

            plot_json = json.dumps(fig, cls=plotly.utils.PlotlyJSONEncoder)
            with open(f'investor-profiles/public/budget_post/{inv_type}_{budget}_contour.json', 'w') as file:
                file.write(plot_json)
        except Exception as error:
            # handle the exception
            print("An exception occurred:", type(error).__name__, "–", error)        
            print("###################################### FAILED TO GENERATE ######################################")


Small 250000
Small 500000
Small 750000
Small 1000000
Small 1250000
Small 1500000
Medium 250000
Medium 500000
Medium 750000
Medium 1000000
Medium 1250000
Medium 1500000
Large 250000
Large 500000
Large 750000
Large 1000000
Large 1250000
Large 1500000
Institutional 250000
Institutional 500000
Institutional 750000
Institutional 1000000
Institutional 1250000
Institutional 1500000


In [32]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KernelDensity




def ellipsoid(x_center, y_center, z_center, width, height, depth, num_points=40):
    # theta ranges from 0 to 2pi and phi from 0 to pi
    theta = np.linspace(0, 2 * np.pi, num_points)
    phi = np.linspace(0, np.pi, num_points)
    theta, phi = np.meshgrid(theta, phi)
    # Correct calculations for the full ellipsoid
    x = x_center + width * np.sin(phi) * np.cos(theta)
    y = y_center + height * np.sin(phi) * np.sin(theta)
    z = z_center + depth * np.cos(phi)
    return x.flatten(),y.flatten(),z.flatten()


for inv_type in inv_color_mapping.keys():
    for budget in [250000, 500000, 750000, 1000000, 1250000, 1500000]:
        print(inv_type, budget)
        try:

            # Filter for a small investor with a specific budget
            df_filtered = df[(df['investor_type_purchase'] == inv_type) & (df['price'] <= budget)]

            # Calculate averages for each style category
            grouped = df_filtered.groupby('style_category').agg({
                'yearbuilt': 'mean',
                'intersf': 'mean',
                'price': 'mean'
            }).reset_index()

            # Normalize data for density estimation
            scaler = StandardScaler()
            X_scaled = scaler.fit_transform(grouped[['yearbuilt', 'intersf', 'price']])

            # Fit KDE to estimate density in the scaled space
            kde = KernelDensity(bandwidth=0.5)
            kde.fit(X_scaled)
            densities = np.exp(kde.score_samples(X_scaled))  # densities are in log space, convert back

            # Define distinct colors for each cluster
            colors = {'Traditional': 'red', 'Modern': 'blue', 'Multi-Unit': 'green'}

            # Calculate standard deviations for each style category across all data points
            std_devs = df_filtered.groupby('style_category')[['yearbuilt', 'intersf', 'price']].std().reset_index()

            # Define max sizes proportionate to standard deviation
            stddev_factor = 0.2
            # middle 15%

            max_sizes = [
                (stddev_factor * std_devs.loc[i, 'yearbuilt'], 
                stddev_factor * std_devs.loc[i, 'intersf'], 
                stddev_factor * std_devs.loc[i, 'price'])
                for i in range(len(std_devs))
            ]

            # Create a 3D scatter plot with density represented by marker size
            fig = go.Figure()

            for i in range(len(grouped)):
                x_center, y_center, z_center = grouped.iloc[i][['yearbuilt', 'intersf', 'price']]
                style_category = grouped.iloc[i]['style_category']
                # Define opacity and size for each ball
                num_balls = 8
                opacities = np.logspace(0.9, 0.01, num_balls)
                # a = np.logspace(0.9, 0.3, num_balls)
                opacities = 0.9*opacities/opacities.max()   
                color = colors[style_category]
                
                # Generate sizes for each axis
                sizes_x = np.linspace(1, max_sizes[i][0], num_balls)
                sizes_y = np.linspace(1, max_sizes[i][1], num_balls)
                sizes_z = np.linspace(1, max_sizes[i][2], num_balls)
                
                for j in range(num_balls):
                    opacity = opacities[j]
                    size_x = sizes_x[j]
                    size_y = sizes_y[j]
                    size_z = sizes_z[j]

                    x, y, z = ellipsoid(x_center, y_center, z_center, size_x, size_y, size_z)

                    # Add ellipsoid to the figure
                    fig.add_trace(go.Mesh3d(
                        name=style_category,
                        x=x,
                        y=y,
                        z=z,
                        color=color,
                        opacity=opacity,
                        alphahull=0
                    ))

            # Add dummy entries for the legend
            for style, color in colors.items():
                fig.add_trace(go.Scatter3d(
                    x=[None],
                    y=[None],
                    z=[None],
                    mode='markers',
                    marker=dict(size=10, color=color),
                    name=style,
                    showlegend=True
                ))


            # Update plot appearance and layout
            fig.update_layout(
                title=f"Historical metadata of different property styles for<br><b>{inv_type}</b> investors with a budget of <b>${budget//1000}k</b>",
                scene=dict(
                    xaxis_title='Year Built',
                    yaxis_title='Internal Square Feet',
                    zaxis_title='Price'
                ),
                margin=dict(l=0, r=0, b=0, t=50)
            )

            # Show plot
            # fig.show()

            plot_json = json.dumps(fig, cls=plotly.utils.PlotlyJSONEncoder)
            with open(f'investor-profiles/public/style_pre/{inv_type}_{budget}_metadata.json', 'w') as file:
                file.write(plot_json)
        except Exception as error:
            # handle the exception
            print("An exception occurred:", type(error).__name__, "–", error)        
            print("###################################### FAILED TO GENERATE ######################################")


Small 250000
Small 500000
Small 750000
Small 1000000
Small 1250000
Small 1500000
Medium 250000
Medium 500000
Medium 750000
Medium 1000000
Medium 1250000
Medium 1500000
Large 250000
Large 500000
Large 750000
Large 1000000
Large 1250000
Large 1500000
Institutional 250000
Institutional 500000
Institutional 750000
Institutional 1000000
Institutional 1250000
Institutional 1500000


In [33]:
import pandas as pd
import numpy as np
from itertools import combinations
import warnings
warnings.filterwarnings("ignore")


# NOTE: NEED TO SCALE THE LINE WIDTH 

for inv_type in inv_color_mapping.keys():
    for budget in [250000, 500000, 750000, 1000000, 1250000, 1500000]:
        for style in ['Traditional', 'Modern', 'Multi-Unit']:
            print(inv_type, budget, style)
            try:
                # Define the mapping of amenity codes to their full names
                amenity_mapping = {
                    'A': 'Porch', 'B': 'Open Porch', 'C': 'Covered Porch', 'D': 'Enclosed Porch', 'E': 'Deck', 
                    'F': 'Terrace/Patio', 'G': 'Balcony', 'H': 'Breezeway', 'I': 'Ingrd Swim Pool', 'J': 'Greenhouse', 
                    'K': 'Tennis Court', 'L': 'Outbuildings', 'M': 'Carriage House', 'O': 'Dock/pier', 
                    'P': 'Waterfront/View', 'Q': 'Attic', 'R': 'Pantry', 'S': 'Den/Library', 'T': 'Recreation Room', 
                    'U': 'Air Conditioned', 'V': 'Sauna', 'W': 'Jacuzzi', 'X': 'Septic System', 'Y': 'Well Water', 
                    'Z': 'Ingrnd Oil Tank', '1': 'Security System', '2': 'Wood Stove', '3': 'Elevator', '4': 'Loading Dock', 
                    '5': 'Irreg Lot Dims', '6': 'Studio', '7': 'Pool', '8': 'Central Vac'
                }

                # Custom categories and colors for amenities
                amenity_categories = {
                    'Leisure': ['Tennis Court', 'Ingrd Swim Pool', 'Greenhouse', 'Jacuzzi', 'Sauna', 'Recreation Room'],
                    'Utility': ['Air Conditioned', 'Septic System', 'Well Water', 'Central Vac', 'Security System', 'Wood Stove', 'Elevator'],
                    'Outdoor': ['Porch', 'Open Porch', 'Covered Porch', 'Enclosed Porch', 'Deck', 'Terrace/Patio', 'Balcony', 'Breezeway', 'Dock/pier'],
                    'Functional': ['Outbuildings', 'Carriage House', 'Loading Dock', 'Studio', 'Pantry', 'Den/Library', 'Attic'],
                    'Luxury': ['Waterfront/View', 'Pool', 'Irreg Lot Dims']
                }

                # Map each amenity to a color
                amenity_color_map = {}
                a_colors = {
                    'Leisure': '#ff7f0e',   # orange
                    'Utility': '#1f77b4',   # blue
                    'Outdoor': '#2ca02c',   # green
                    'Functional': '#d62728',# red
                    'Luxury': '#9467bd'     # purple
                }

                for category, amenities in amenity_categories.items():
                    for amenity in amenities:
                        amenity_color_map[amenity] = a_colors[category]

                # Filter the DataFrame
                df_filtered = df[
                    (df['investor_type_purchase'] == inv_type) & 
                    (df['price'] <= budget) & 
                    (df['style_category'] == style)
                ]

                # Function to parse amenities
                def parse_amenities(amenity_string):
                    if pd.isna(amenity_string):
                        return []
                    return [amenity_mapping[char] for char in amenity_string if char in amenity_mapping]

                # Apply parsing function to the 'amenity' column
                df_filtered['amenities'] = df_filtered['amenity'].apply(parse_amenities)

                all_amenities = list(amenity_mapping.values())
                adjacency_matrix = pd.DataFrame(np.zeros((len(all_amenities), len(all_amenities))), index=all_amenities, columns=all_amenities)

                # Populate the adjacency matrix
                for amenities in df_filtered['amenities']:
                    for (a1, a2) in combinations(set(amenities), 2):
                        adjacency_matrix.at[a1, a2] += 1
                        adjacency_matrix.at[a2, a1] += 1

                # Ensure no NaN values and data type is integer
                adjacency_matrix.fillna(0, inplace=True)
                adjacency_matrix = adjacency_matrix.astype(int)

                # Remove amenities that do not co-occur with others
                adjacency_matrix = adjacency_matrix.loc[adjacency_matrix.sum(axis=1) > 0, adjacency_matrix.sum(axis=0) > 0]

                # Update 'all_amenities' to match filtered matrix
                all_amenities = adjacency_matrix.index.tolist()

                # Function to generate points for Bezier curves between nodes
                def get_bezier_curve(x1, y1, x2, y2, num_points=20):
                    # Control points for Bezier curve
                    cx1 = (x1 + x2) / 2
                    cy1 = y1
                    cx2 = (x1 + x2) / 2
                    cy2 = y2
                    
                    # Generate points along the Bezier curve
                    t = np.linspace(0, 1, num_points)
                    x = (1 - t)**3 * x1 + 3 * (1 - t)**2 * t * cx1 + 3 * (1 - t) * t**2 * cx2 + t**3 * x2
                    y = (1 - t)**3 * y1 + 3 * (1 - t)**2 * t * cy1 + 3 * (1 - t) * t**2 * cy2 + t**3 * y2
                    return x, y

                # Assuming 'adjacency_matrix' is defined and correct
                amenities = adjacency_matrix.index.tolist()
                num_amenities = len(amenities)
                angle = np.linspace(0, 2 * np.pi, num_amenities, endpoint=False)  # Node positions on circle
                x = np.cos(angle)
                y = np.sin(angle)


                fig = go.Figure()
                min_node = 10
                max_node = 30
                gradient = (max_node-min_node)/adjacency_matrix.sum().max()

                node_sizes = adjacency_matrix.sum().apply(lambda x: min_node + x *gradient)  # Scale node size


                # Draw Bezier curves for each connection
                for i, start in enumerate(all_amenities):
                    for j, end in enumerate(all_amenities):
                        if i != j and adjacency_matrix.at[start, end] > 0:
                            x_bez, y_bez = get_bezier_curve(x[i], y[i], x[j], y[j])
                            strength = adjacency_matrix.at[start, end]
                            fig.add_trace(go.Scatter(
                                x=x_bez, y=y_bez, mode='lines',
                                line=dict(width=strength * 0.1 + 1, color=col_to_fillcol(inv_color_mapping[inv_type], alpha = 0.2)),
                                hoverinfo='text',
                                text=f"Contains {start} and {end}: {strength}",
                                showlegend=False))

                # Add nodes as scatter points with hover information
                fig.add_trace(go.Scatter(
                    x=x, y=y, mode='markers+text', text=all_amenities,
                    marker=dict(size=node_sizes, color='black'),
                    textposition='top center',
                    hoverinfo='text',
                    hovertext=[f"{amenity}: {adjacency_matrix.sum()[amenity]}" for amenity in all_amenities]
                ))

                # Set plot layout
                fig.update_layout(showlegend=False, xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                                yaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                                plot_bgcolor='white',
                                title=f"Network Diagram of Amenities for <b>{style}</b><br> style homes under a "+\
                                f"<b>${budget//1000}k</b> budget for <b>{inv_type}</b> investors")

                # fig.show()



                plot_json = json.dumps(fig, cls=plotly.utils.PlotlyJSONEncoder)
                with open(f'investor-profiles/public/style_post/{inv_type}_{budget}_{style}_network.json', 'w') as file:
                    file.write(plot_json)
            except Exception as error:
                # handle the exception
                print("An exception occurred:", type(error).__name__, "–", error)        
                print("###################################### FAILED TO GENERATE ######################################")


Small 250000 Traditional
Small 250000 Modern
Small 250000 Multi-Unit
Small 500000 Traditional
Small 500000 Modern
Small 500000 Multi-Unit
Small 750000 Traditional
Small 750000 Modern
Small 750000 Multi-Unit
Small 1000000 Traditional
Small 1000000 Modern
Small 1000000 Multi-Unit
Small 1250000 Traditional
Small 1250000 Modern
Small 1250000 Multi-Unit
Small 1500000 Traditional
Small 1500000 Modern
Small 1500000 Multi-Unit
Medium 250000 Traditional
Medium 250000 Modern
Medium 250000 Multi-Unit
Medium 500000 Traditional
Medium 500000 Modern
Medium 500000 Multi-Unit
Medium 750000 Traditional
Medium 750000 Modern
Medium 750000 Multi-Unit
Medium 1000000 Traditional
Medium 1000000 Modern
Medium 1000000 Multi-Unit
Medium 1250000 Traditional
Medium 1250000 Modern
Medium 1250000 Multi-Unit
Medium 1500000 Traditional
Medium 1500000 Modern
Medium 1500000 Multi-Unit
Large 250000 Traditional
Large 250000 Modern
Large 250000 Multi-Unit
Large 500000 Traditional
Large 500000 Modern
Large 500000 Multi-Uni

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt

# Assuming df is your DataFrame containing geometry information

# Choose the subregion you want to visualize
subregion_to_visualize = 'SWAP'  # Change this to the subregion you want to visualize

# Filter the DataFrame to select only the rows corresponding to the chosen subregion
subregion_df = df[df['subregion'] == subregion_to_visualize]

print(len(subregion_df))

# Convert the filtered DataFrame to a GeoDataFrame
subregion_gdf = gpd.GeoDataFrame(subregion_df)

# Plot the geometry of the selected subregion
fig, ax = plt.subplots(figsize=(10, 8))
subregion_gdf.plot(ax=ax, color='blue', edgecolor='black')
ax.set_title(f'Subregion: {subregion_to_visualize} Visualization')
plt.show()


In [ ]:
from shapely.ops import unary_union
from tqdm import tqdm
# Assuming df is your DataFrame containing geometry information

# Create a dictionary to store the geometry for each subregion
subregion_geometry_dict = {}

# Iterate over unique subregions
for subregion in tqdm(df['subregion'].unique()):
    # Filter the DataFrame for the current subregion
    subregion_df = df[df['subregion'] == subregion]
    
    # Combine the geometries for the current subregion into a single geometry
    subregion_geometry = unary_union(subregion_df['geometry'])
    
    # Store the combined geometry in the dictionary
    subregion_geometry_dict[subregion] = subregion_geometry

# Create a new column "geometry_subregion" by mapping the subregion to its combined geometry
df['geometry_subregion'] = df['subregion'].map(subregion_geometry_dict)

In [35]:
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from scipy.stats import gaussian_kde
from scipy.interpolate import CubicSpline


for subregion in list(df['subregion'].unique()):
    print(subregion)
    try:

        # Initialize a subplot object with specified subplot types
        fig = make_subplots(
            rows=2, cols=3,
            specs=[[{'type': 'pie'}, {'type': 'xy'}, {'type': 'xy'}],
                [{'type': 'xy'}, {'type': 'xy'}, {'type': 'xy'}]],
            subplot_titles=('Race Composition', 'Age Distribution', 'Median Household Income ($)',
                            'Household Composition', 'Median Income by Tenure', 'Gender Distribution')
        )

        # Define the investment criteria and subregion

        # Filter the DataFrame for the investor's criteria and subregion
        df_filtered = df[
            (df['subregion'] == subregion)
        ]


        # Assuming df_filtered now contains the demographics data for the specified subregion
        # Proceed to create visualizations...

        # Race Composition (Pie Chart)
        race_data = df_filtered[['nhwhi', 'nhaa', 'nhna', 'nhas', 'nhpi', 'nhapi', 'nhoth', 'nhmlt', 'latin']].sum()
        race_labels = ['White', 'Black or African American', 'American Indian', 'Asian', 'Pacific Islander',
                    'Asian and Pacific Islander', 'Other Race', 'Multiracial', 'Hispanic or Latino']
        pie = go.Pie(labels=race_labels, values=race_data.values, name="Race Composition")
        fig.add_trace(pie, row=1, col=1)

        # Turn off legends for all traces except the pie chart
        for i, trace in enumerate(fig.data):
            if trace.type != "pie":
                trace.showlegend = False

        # Age Distribution (Bar Chart)
        age_data = df_filtered[['pop_u18', 'pop_18_64', 'pop_65o']].sum()
        age_labels = ['Under 18', '18 - 64', '65+']
        bar_age = go.Bar(x=age_labels, y=age_data.values, name="Age Distribution", showlegend=False)
        fig.add_trace(bar_age, row=1, col=2)

        # Median Household Income (Histogram)
        # Compute the KDE using Gaussian kernel
        num_bins = 20

        # Compute the density values
        density, bins = np.histogram(df_filtered['mhi'], bins=num_bins, density=True)
        bin_centers = (bins[:-1] + bins[1:]) / 2
        spline_interpolation = CubicSpline(bin_centers, density)
        x_values = np.linspace(min(df_filtered['mhi']), max(df_filtered['mhi']), 500)
        hist, _ = np.histogram(df_filtered['mhi'], bins=num_bins, density=False)
        density_values = spline_interpolation(x_values)
        density_values = density_values * max(hist) / max(density_values)
        # Filter out negative density values
        density_values = np.maximum(density_values, 0)

        hist_mhi = go.Histogram(x=df_filtered['mhi'], name="Household Income", showlegend=False, nbinsx=num_bins, opacity = 0.5)
        density_trace = go.Scatter(
            x=x_values,
            y=density_values,
            mode='lines',
            name='Density',
            line=dict(color='red', width=2),
            showlegend=False
        )

        fig.add_trace(hist_mhi, row=1, col=3)
        fig.add_trace(density_trace, row=1, col=3)

        # Household Composition by Size (Stacked Bar Chart)
        hh_size_data = df_filtered[['fhh2', 'fhh3', 'fhh4', 'fhh5', 'fhh6', 'fhh7o',
                                    'nfhh1', 'nfhh2', 'nfhh3', 'nfhh4', 'nfhh5', 'nfhh6', 'nfhh7o']].sum()
        hh_size_labels = ['2-Person', '3-Person', '4-Person', '5-Person', '6-Person', '7+-Person',
                        '1-Person Nonfamily', '2-Person Nonfamily', '3-Person Nonfamily', '4-Person Nonfamily',
                        '5-Person Nonfamily', '6-Person Nonfamily', '7+-Person Nonfamily']
        bar_hh = go.Bar(x=hh_size_labels, y=hh_size_data.values, name="Household Composition", showlegend=False)
        fig.add_trace(bar_hh, row=2, col=1)

        # Median Household Income by Tenure (Box Plot)
        box_owner = go.Box(y=df_filtered['o_mhi'], name='Owner-Occupied', showlegend=False)
        box_renter = go.Box(y=df_filtered['r_mhi'], name='Renter-Occupied', showlegend=False)
        fig.add_trace(box_owner, row=2, col=2)
        fig.add_trace(box_renter, row=2, col=2)

        # Gender and Age (Scatter Plot)
        gender_data = [df_filtered['male'].mean(), df_filtered['female'].mean()]
        gender_labels = ['Male', 'Female']
        bar_gender = go.Bar(x=gender_labels, y=gender_data, name="Gender Distribution")
        fig.add_trace(bar_gender, row=2, col=3)

        # Update layout with annotation for the binary gender data note
        fig.add_annotation(
            text="<b>Note:</b> Gender data is binary as dataset only contained binary labels",
            xref="paper", yref="paper",
            x=1.03, y=-0.1, showarrow=False,
            xanchor='right', yanchor='bottom',
            font=dict(size=12),
            align='left', bgcolor='rgba(255,255,255,0.9)',
            bordercolor='black', borderwidth=1
        )

        # Customize layout
        fig.update_layout(
            # Set the height, width, and title as needed
            height=1000,
            width=1200,
            title_text=f"Demographics Dashboard for <b>'{subregion}'</b> MAPC Subregion",
        )
        #  Update pie chart legend to be inside its subplot
        # fig.update_traces(marker=dict(line=dict(color='#000000', width=1)), selector=dict(type='pie'))
        fig.update_layout(
            legend=dict(
                title=dict(text='Race Composition'),
                orientation='h',
                yanchor="bottom",
                y=0.45,
                xanchor="center",
                x=0.5,
                bordercolor="black",  # Choose the border color
                borderwidth=1       
            )
        )

        # fig.update_layout(
        #     margin=dict(l=30, r=50, t=200, b=100),  # Adjust overall margin to create space for borders

        # )


        # Show the dashboard
        # fig.show()
        plot_json = json.dumps(fig, cls=plotly.utils.PlotlyJSONEncoder)
        with open(f'investor-profiles/public/loc_post/{subregion}_dashboard.json', 'w') as file:
            file.write(plot_json)
    except Exception as error:
        # handle the exception
        print("An exception occurred:", type(error).__name__, "–", error)        
        print("###################################### FAILED TO GENERATE ######################################")


TRIC
MAGIC
ICC
MWRC
SSC
NSPC
SWAP
NSTF


In [36]:
import plotly.graph_objs as go
import numpy as np


for inv_type in inv_color_mapping.keys():
    for budget in [250000, 500000, 750000, 1000000, 1250000, 1500000]:
        for style in ['Traditional', 'Modern', 'Multi-Unit']:
            for subregion in list(df['subregion'].unique()):
                print(inv_type, budget, style, subregion)
                try:

                    # Filter the DataFrame for the investor's criteria and subregion
                    df_filtered = df[
                        (df['investor_type_purchase'] == inv_type) &
                        (df['price'] <= budget) &
                        (df['style_category'] == style) &
                        (df['subregion'] == subregion)
                    ]

                    # Map 'buy_side_flip' to 1 for 'Yes', 0 for 'No'
                    flip_data = df_filtered['buy_side_flip'].map({1: 1, 0: 0}).dropna()

                    # Calculate percentages
                    total_count = flip_data.count()
                    flipped_count = int(100 * (flip_data.sum() / total_count))
                    not_flipped_count = 100 - flipped_count

                    # Define the size of the matrix (always 10x10 for 100 dots)
                    matrix_size = 10

                    # Generate the coordinates
                    x_values, y_values = np.meshgrid(range(matrix_size), range(matrix_size))
                    x_values = x_values.flatten()
                    y_values = y_values.flatten()

                    # Determine the indices for flipped and not flipped properties
                    flipped_indices = np.arange(flipped_count)
                    not_flipped_indices = np.arange(flipped_count, 100)

                    flipped_markers = go.Scatter(
                        x=x_values[flipped_indices],
                        y=y_values[flipped_indices],
                        mode='markers',
                        name='Flipped',
                        marker=dict(color='red', size=30),
                        hoverinfo='text',
                        text='Flipped'
                    )

                    not_flipped_markers = go.Scatter(
                        x=x_values[not_flipped_indices],
                        y=y_values[not_flipped_indices],
                        mode='markers',
                        name='Not Flipped',
                        marker=dict(color='rgba(0,128,0,0.8)', size=30),
                        hoverinfo='text',
                        text='Not Flipped'
                    )

                    fig = go.Figure(data=[not_flipped_markers, flipped_markers])

                    fig.add_annotation(
                        text=f"<b>{flipped_count}%</b>",
                        xref="paper", yref="paper",
                        x=0.6, y=-0.1, showarrow=False,
                        xanchor='left', yanchor='bottom',
                        font=dict(size=40),
                        align='left', bgcolor='rgba(255,255,255,0.9)',
                    )

                    # Update layout
                    fig.update_layout(
                        title=f"How often do investors with your choices flip properties?" + \
                        f"<i><br><b>{inv_type}</b> investor, <b>${budget//1000}k</b> budget, <br>" + \
                        f"<b>{style}</b> property style, <b>{subregion}</b> MAPC subregion</i>",
                        xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                        yaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                        width=600,
                        height=600,
                        legend=dict(x=0.1, y=-0.1, bordercolor="Black", borderwidth=1),
                        plot_bgcolor='white'
                    )

                    # Show plot
                    # fig.show()

                    plot_json = json.dumps(fig, cls=plotly.utils.PlotlyJSONEncoder)
                    with open(f'investor-profiles/public/flip_pre/{inv_type}_{budget}_{style}_{subregion}_dots.json', 'w') as file:
                        file.write(plot_json)
                except Exception as error:
                    # handle the exception
                    print("An exception occurred:", type(error).__name__, "–", error)        
                    print("###################################### FAILED TO GENERATE ######################################")


Small 250000 Traditional TRIC
Small 250000 Traditional MAGIC
Small 250000 Traditional ICC
Small 250000 Traditional MWRC
Small 250000 Traditional SSC
Small 250000 Traditional NSPC
Small 250000 Traditional SWAP
Small 250000 Traditional NSTF
Small 250000 Modern TRIC
Small 250000 Modern MAGIC
Small 250000 Modern ICC
Small 250000 Modern MWRC
Small 250000 Modern SSC
Small 250000 Modern NSPC
Small 250000 Modern SWAP
Small 250000 Modern NSTF
Small 250000 Multi-Unit TRIC
Small 250000 Multi-Unit MAGIC
Small 250000 Multi-Unit ICC
Small 250000 Multi-Unit MWRC
Small 250000 Multi-Unit SSC
Small 250000 Multi-Unit NSPC
Small 250000 Multi-Unit SWAP
Small 250000 Multi-Unit NSTF
Small 500000 Traditional TRIC
Small 500000 Traditional MAGIC
Small 500000 Traditional ICC
Small 500000 Traditional MWRC
Small 500000 Traditional SSC
Small 500000 Traditional NSPC
Small 500000 Traditional SWAP
Small 500000 Traditional NSTF
Small 500000 Modern TRIC
Small 500000 Modern MAGIC
Small 500000 Modern ICC
Small 500000 Mode

In [37]:
import plotly.graph_objs as go
from plotly.subplots import make_subplots

for inv_type in inv_color_mapping.keys():
    for budget in [250000, 500000, 750000, 1000000, 1250000, 1500000]:
        for style in ['Traditional', 'Modern', 'Multi-Unit']:
            for subregion in list(df['subregion'].unique()):
                print(inv_type, budget, style, subregion)
                try:

                    # Filter the DataFrame for the investor's criteria
                    df_filtered = df[
                        (df['investor_type_sale'] == inv_type) &
                        (df['price'] <= budget) &
                        (df['style_category'] == style) &
                        (df['subregion'] == subregion) &        
                        (df['flip_horizon'].notna())
                    ]

                    # Convert 'flip_horizon' from days to months
                    df_filtered['flip_horizon_months'] = df_filtered['flip_horizon'] / 30.0
                    lower_bound = df_filtered['flip_horizon_months'].quantile(0.025)
                    upper_bound = df_filtered['flip_horizon_months'].quantile(0.975)

                    # Filter the DataFrame to include only the middle 95% of flip durations
                    df_filtered = df_filtered[
                        (df_filtered['flip_horizon_months'] >= lower_bound) &
                        (df_filtered['flip_horizon_months'] <= upper_bound)
                    ]


                    # Filter investors who flip and those who do not
                    flippers = df_filtered[df_filtered['sell_side_flip'] == 1]
                    non_flippers = df_filtered[df_filtered['sell_side_flip'] == 0]

                    # Create subplot with 1 row and 2 columns
                    fig = make_subplots(
                        rows=1, cols=2,
                        subplot_titles=('Flippers', 'Non-Flippers'),
                        specs=[[{}, {}]],  # Standard Cartesian plots
                        horizontal_spacing=0.15  # Adjust spacing to emphasize separation
                    )

                    # Add box plot for flippers on the first subplot
                    fig.add_trace(go.Box(
                        y=flippers['flip_horizon_months'],
                        name='Flippers',
                        boxpoints='all',
                        jitter=0.5,
                        pointpos=-2,
                        marker_color='red'
                    ), row=1, col=1)

                    # Add box plot for non-flippers on the second subplot
                    fig.add_trace(go.Box(
                        y=non_flippers['flip_horizon_months'],
                        name='Non-Flippers',
                        boxpoints='all',
                        jitter=0.5,
                        pointpos=2,
                        marker_color='green'
                    ), row=1, col=2)

                    # Update layout
                    fig.update_layout(
                        title=f"Property Holding Period by <b>{inv_type}</b> Investor Flipping Activity",
                        yaxis=dict(
                            title="Holding Period (Months) - Flippers",
                            gridcolor='rgba(150, 150, 150, 0.5)',
                            zerolinecolor='rgba(150, 150, 150, 0.8)'
                        ),
                        yaxis2=dict(
                            title="Holding Period (Months) - Non-Flippers",
                            gridcolor='rgba(150, 150, 150, 0.5)',
                            zerolinecolor='rgba(150, 150, 150, 0.8)'
                        ),
                        showlegend=False,
                        plot_bgcolor='rgba(243, 243, 243, 0.5)',
                        width=800,
                        height=600
                    )

                    # Show plot
                    # fig.show()

                    plot_json = json.dumps(fig, cls=plotly.utils.PlotlyJSONEncoder)
                    with open(f'investor-profiles/public/flip_post/{inv_type}_{budget}_{style}_{subregion}_holding_period.json', 'w') as file:
                        file.write(plot_json)
                except Exception as error:
                    # handle the exception
                    print("An exception occurred:", type(error).__name__, "–", error)        
                    print("###################################### FAILED TO GENERATE ######################################")


Small 250000 Traditional TRIC
Small 250000 Traditional MAGIC
Small 250000 Traditional ICC
Small 250000 Traditional MWRC
Small 250000 Traditional SSC
Small 250000 Traditional NSPC
Small 250000 Traditional SWAP
Small 250000 Traditional NSTF
Small 250000 Modern TRIC
Small 250000 Modern MAGIC
Small 250000 Modern ICC
Small 250000 Modern MWRC
Small 250000 Modern SSC
Small 250000 Modern NSPC
Small 250000 Modern SWAP
Small 250000 Modern NSTF
Small 250000 Multi-Unit TRIC
Small 250000 Multi-Unit MAGIC
Small 250000 Multi-Unit ICC
Small 250000 Multi-Unit MWRC
Small 250000 Multi-Unit SSC
Small 250000 Multi-Unit NSPC
Small 250000 Multi-Unit SWAP
Small 250000 Multi-Unit NSTF
Small 500000 Traditional TRIC
Small 500000 Traditional MAGIC
Small 500000 Traditional ICC
Small 500000 Traditional MWRC
Small 500000 Traditional SSC
Small 500000 Traditional NSPC
Small 500000 Traditional SWAP
Small 500000 Traditional NSTF
Small 500000 Modern TRIC
Small 500000 Modern MAGIC
Small 500000 Modern ICC
Small 500000 Mode

In [38]:
import plotly.graph_objs as go
from plotly.subplots import make_subplots

# Define the investment criteria
# Define the investment criteria
for inv_type in inv_color_mapping.keys():
    for budget in [250000, 500000, 750000, 1000000, 1250000, 1500000]:
        for style in ['Traditional', 'Modern', 'Multi-Unit']:
            for subregion in list(df['subregion'].unique()):
                print(inv_type, budget, style, subregion)
                try:

                    # Filter the DataFrame for the investor's criteria
                    df_filtered = df[
                        (df['investor_type_sale'] == inv_type) &
                        (df['price'] <= budget) &
                        (df['style_category'] == style) &
                        (df['subregion'] == subregion) &        
                        (df['price_diff'].notna())
                    ]

                    lower_bound = df_filtered['price_diff'].quantile(0.025)
                    upper_bound = df_filtered['price_diff'].quantile(0.975)

                    # Filter the DataFrame to include only the middle 95% of flip durations
                    df_filtered = df_filtered[
                        (df_filtered['price_diff'] >= lower_bound) &
                        (df_filtered['price_diff'] <= upper_bound)
                    ]


                    # Filter investors who flip and those who do not
                    flippers = df_filtered[df_filtered['sell_side_flip'] == 1]
                    non_flippers = df_filtered[df_filtered['sell_side_flip'] == 0]

                    # Create subplot with 1 row and 2 columns
                    fig = make_subplots(
                        rows=1, cols=2,
                        subplot_titles=('Profits of Flippers', 'Profits of Non-Flippers'),
                        specs=[[{}, {}]],  # Standard Cartesian plots
                        horizontal_spacing=0.15  # Adjust spacing to emphasize separation
                    )

                    # Add violin plot for flippers on the first subplot
                    fig.add_trace(go.Violin(
                        y=flippers['price_diff'],
                        name='Flippers',
                        box_visible=True,
                        meanline_visible=True,
                        line_color='red',
                        fillcolor='red',
                        opacity=0.6,
                        box=dict(
                            visible=True,
                            line=dict(
                                color='black'  # Ensuring the box lines are clearly visible
                            ),
                            fillcolor='black'  # Optional, can set a different fill color for the box if needed
                        ),
                        points='all'  # Show all points
                    ), row=1, col=1)

                    # Add violin plot for non-flippers on the second subplot
                    fig.add_trace(go.Violin(
                        y=non_flippers['price_diff'],
                        name='Non-Flippers',
                        box_visible=True,
                        meanline_visible=True,
                        line_color='green',
                        fillcolor='green',
                        opacity=0.6,
                        points='all',
                        box=dict(
                            visible=True,
                            line=dict(
                                color='black'  # Ensuring the box lines are clearly visible
                            ),
                            fillcolor='black'  # Optional, can set a different fill color for the box if needed
                        ),
                    ), row=1, col=2)

                    # Update layout
                    fig.update_layout(
                        title=f"Profit Distribution by Investor Flipping Activity based on your choices." +\
                        f"<i><br><b>{inv_type}</b> investor, <b>${budget//1000}k</b> budget, " + \
                        f"<b>{style}</b> property style, <b>{subregion}</b> MAPC subregion</i>",
                        yaxis=dict(
                            title="Profit - Flippers",
                            gridcolor='rgba(150, 150, 150, 0.5)',
                            zerolinecolor='rgba(150, 150, 150, 0.8)'
                        ),
                        yaxis2=dict(
                            title="Profit - Non-Flippers",
                            gridcolor='rgba(150, 150, 150, 0.5)',
                            zerolinecolor='rgba(150, 150, 150, 0.8)'
                        ),
                        showlegend=False,
                        plot_bgcolor='rgba(243, 243, 243, 0.5)',
                        width=800,
                        height=600
                    )

                    # Show plot
                    # fig.show()

                    plot_json = json.dumps(fig, cls=plotly.utils.PlotlyJSONEncoder)
                    with open(f'investor-profiles/public/flip_post/{inv_type}_{budget}_{style}_{subregion}_profit.json', 'w') as file:
                        file.write(plot_json)
                except Exception as error:
                    # handle the exception
                    print("An exception occurred:", type(error).__name__, "–", error)        
                    print("###################################### FAILED TO GENERATE ######################################")


Small 250000 Traditional TRIC
Small 250000 Traditional MAGIC
Small 250000 Traditional ICC
Small 250000 Traditional MWRC
Small 250000 Traditional SSC
Small 250000 Traditional NSPC
Small 250000 Traditional SWAP
Small 250000 Traditional NSTF
Small 250000 Modern TRIC
Small 250000 Modern MAGIC
Small 250000 Modern ICC
Small 250000 Modern MWRC
Small 250000 Modern SSC
Small 250000 Modern NSPC
Small 250000 Modern SWAP
Small 250000 Modern NSTF
Small 250000 Multi-Unit TRIC
Small 250000 Multi-Unit MAGIC
Small 250000 Multi-Unit ICC
Small 250000 Multi-Unit MWRC
Small 250000 Multi-Unit SSC
Small 250000 Multi-Unit NSPC
Small 250000 Multi-Unit SWAP
Small 250000 Multi-Unit NSTF
Small 500000 Traditional TRIC
Small 500000 Traditional MAGIC
Small 500000 Traditional ICC
Small 500000 Traditional MWRC
Small 500000 Traditional SSC
Small 500000 Traditional NSPC
Small 500000 Traditional SWAP
Small 500000 Traditional NSTF
Small 500000 Modern TRIC
Small 500000 Modern MAGIC
Small 500000 Modern ICC
Small 500000 Mode

In [40]:
import plotly.graph_objects as go
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors  # Ensure this import is correct

for inv_type in inv_color_mapping.keys():
    for budget in [250000, 500000, 750000, 1000000, 1250000, 1500000]:
        for style in ['Traditional', 'Modern', 'Multi-Unit']:
            for subregion in list(df['subregion'].unique()):
                for flip_var in ['NoFlip', 'Flip']:
                    print(inv_type, budget, style, subregion, flip_var)
                    try:

                        # Remove rows with null values in the relevant columns and filter out 'Non-investor'
                        df_cleaned = df.dropna(subset=['investor_type_purchase', 'style_category', 'subregion', 'buy_side_flip'])
                        df_cleaned = df_cleaned[df_cleaned['investor_type_purchase'] != 'Non-investor']

                        # Map 'buy_side_flip' from 0, 1 to 'NoFlip', 'Flip'
                        df_cleaned['flip_status'] = df_cleaned['buy_side_flip'].map({0: 'NoFlip', 1: 'Flip'})

                        # Generate color mappings for each category
                        def generate_color_map(categories):
                            cmap = plt.get_cmap('tab20')  # You can choose any matplotlib colormap that fits your aesthetic
                            colors = cmap(np.linspace(0, 1, len(categories)))
                            hex_colors = [mcolors.rgb2hex(color) for color in colors]
                            return dict(zip(categories, hex_colors))

                        budget_thresholds = [250000, 500000, 750000, 1000000, 1250000, 1500000]
                        # Generate color mappings for budgets
                        budget_colors = plt.get_cmap('Greys')(np.linspace(0, 1, len(budget_thresholds)))
                        budget_color_mapping = {budget: mcolors.rgb2hex(color) for budget, color in zip(budget_thresholds, budget_colors)}

                        # Unique identifiers for each category stage
                        inv_types = df_cleaned['investor_type_purchase'].unique()
                        styles = df_cleaned['style_category'].unique()
                        subregions = df_cleaned['subregion'].unique()
                        flips = df_cleaned['flip_status'].unique()

                        style_color_mapping = generate_color_map(styles)
                        subregion_color_mapping = generate_color_map(subregions)
                        flip_color_mapping = {'Flip': 'orange', 'NoFlip': 'lightblue'}

                        # Combine all mappings
                        label_to_color = {**inv_color_mapping,
                                        **style_color_mapping,
                                        **subregion_color_mapping,
                                        **flip_color_mapping,
                                        **budget_color_mapping}

                        # Extending node labels for budgets
                        all_nodes = np.concatenate((inv_types, budget_thresholds, styles, subregions, flips))
                        node_labels = list(dict.fromkeys(all_nodes))  # Remove duplicates
                        label_to_index = {label: idx for idx, label in enumerate(node_labels)}

                        # Initialize links
                        links = {
                            'source': [],
                            'target': [],
                            'value': []
                        }

                        # Function to add links
                        def add_links(df, source_col, target_col):
                            counts = df.groupby([source_col, target_col]).size().reset_index(name='count')
                            for _, row in counts.iterrows():
                                source_index = label_to_index[row[source_col]]
                                target_index = label_to_index[row[target_col]]
                                links['source'].append(source_index)
                                links['target'].append(target_index)
                                links['value'].append(row['count'])

                        # Helper to directly add links based on provided names
                        def add_links_directly(source, target, value):
                            links['source'].append(label_to_index[source])
                            links['target'].append(label_to_index[target])
                            links['value'].append(value)

                        # Adding initial links from investor type to budgets
                        for inv_type_ in inv_types:
                            for budget_ in budget_thresholds:
                                budget_value = df_cleaned[(df_cleaned['investor_type_purchase'] == inv_type_) & (df_cleaned['price'] <= budget_)]['price'].count()
                                if budget_value > 0:
                                    add_links_directly(inv_type_, budget_, budget_value)

                        # Reconnecting from budget to style category
                        for budget_ in budget_thresholds:
                            for style_ in styles:
                                style_value = df_cleaned[(df_cleaned['price'] <= budget_) & (df_cleaned['style_category'] == style_)]['price'].count()
                                if style_value > 0:
                                    add_links_directly(budget_, style_, style_value)

                        # Continue with original link logic
                        add_links(df_cleaned, 'style_category', 'subregion')
                        add_links(df_cleaned, 'subregion', 'flip_status')

                        # Define the path to highlight
                        highlight_path = [(inv_type, budget), 
                                        (budget, style), 
                                        (style, subregion), 
                                        (subregion, flip_var)]

                        # Initialize link colors
                        link_colors = ['rgba(128,128,128,0.5)'] * len(links['source'])

                        # Function to set link colors for the highlight path
                        def set_highlight_path(links, highlight_path, color='red'):
                            for source, target in highlight_path:
                                for i in range(len(links['source'])):
                                    if node_labels[links['source'][i]] == source and node_labels[links['target'][i]] == target:
                                        link_colors[i] = color

                        # Set link colors for the highlight path
                        set_highlight_path(links, highlight_path)



                        final_df = df_cleaned[
                            (df_cleaned['investor_type_sale'] == inv_type) &
                            (df_cleaned['price'] <= budget) &
                            (df_cleaned['style_category'] == style) &
                            (df_cleaned['subregion'] == subregion) &        
                            (df_cleaned['flip_status'] == flip_var)
                        ]

                        perc_ = 100 * len(final_df)/len(df_cleaned)

                        # Create and display the Sankey diagram with customized link colors
                        fig = go.Figure(data=[go.Sankey(
                            node=dict(
                                pad=15,
                                thickness=20,
                                line=dict(color="black", width=0.9),
                                label=[str(label) for label in node_labels],  # Convert all labels to string to ensure proper display
                                color=[label_to_color[label] for label in node_labels]  # Apply colors to nodes
                            ),
                            link=dict(
                                source=links['source'],
                                target=links['target'],
                                value=links['value'],
                                color=link_colors  # Set custom link colors
                            )
                        )])

                        fig.add_annotation(
                            text=f"You took the same path as <b>{perc_:.2f}%</b> of property investors in Greater Boston</b>",
                            xref="paper", yref="paper",
                            x=0., y=-0.2, showarrow=False,
                            xanchor='left', yanchor='bottom',
                            font=dict(size=12),
                            align='left', bgcolor='rgba(255,255,255,0.9)',
                        )

                        fig.update_layout(title_text="Sankey Diagram of Investment Choices with your path highlighter in <span style='color:red'><b>red</b></span>", font_size=12)
                        # fig.show()

                        plot_json = json.dumps(fig, cls=plotly.utils.PlotlyJSONEncoder)
                        with open(f'investor-profiles/public/summary/{inv_type}_{budget}_{style}_{subregion}_{flip_var}_sankey.json', 'w') as file:
                            file.write(plot_json)
                    except Exception as error:
                        # handle the exception
                        print("An exception occurred:", type(error).__name__, "–", error)        
                        print("###################################### FAILED TO GENERATE ######################################")


Small 250000 Traditional TRIC NoFlip
Small 250000 Traditional TRIC Flip
Small 250000 Traditional MAGIC NoFlip
Small 250000 Traditional MAGIC Flip
Small 250000 Traditional ICC NoFlip
Small 250000 Traditional ICC Flip
Small 250000 Traditional MWRC NoFlip
Small 250000 Traditional MWRC Flip
Small 250000 Traditional SSC NoFlip
Small 250000 Traditional SSC Flip
Small 250000 Traditional NSPC NoFlip
Small 250000 Traditional NSPC Flip
Small 250000 Traditional SWAP NoFlip
Small 250000 Traditional SWAP Flip
Small 250000 Traditional NSTF NoFlip
Small 250000 Traditional NSTF Flip
Small 250000 Modern TRIC NoFlip
Small 250000 Modern TRIC Flip
Small 250000 Modern MAGIC NoFlip
Small 250000 Modern MAGIC Flip
Small 250000 Modern ICC NoFlip
Small 250000 Modern ICC Flip
Small 250000 Modern MWRC NoFlip
Small 250000 Modern MWRC Flip
Small 250000 Modern SSC NoFlip
Small 250000 Modern SSC Flip
Small 250000 Modern NSPC NoFlip
Small 250000 Modern NSPC Flip
Small 250000 Modern SWAP NoFlip
Small 250000 Modern SWAP

In [41]:
df['flip_status'] = df['buy_side_flip'].map({0: 'NoFlip', 1: 'Flip'})

In [51]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from scipy.interpolate import make_interp_spline, interp1d
from scipy.integrate import simps
from scipy.interpolate import interp1d

# just for this once


for inv_type in inv_color_mapping.keys():
    print(inv_type)
    try:

        # CUSTOM DFINITION
        def choice_condition(x):
            return ((x['investor_type_purchase'] == inv_type))

        def total_condition(x):
            return (x['investor_type_purchase'].isin(list(inv_color_mapping.keys())))

        year_gap = 5
        custom_choice_text = f'{inv_type} investor'

        # Assuming df is your DataFrame after the join and has been loaded correctly
        # Filter the resulting DataFrame to the middle 95% of the 'price_change_percentage' data
        lower_bound, upper_bound = np.percentile(df['price'], [2.5, 97.5])
        filtered_df = df[(df['price'] >= lower_bound) & (df['price'] <= upper_bound)]
        global_fraction = filtered_df[choice_condition(filtered_df)].shape[0] / len(filtered_df)

        def custom_counter(geoid_data):
            choice_count = geoid_data[choice_condition(geoid_data)].shape[0]
            total_count = geoid_data[total_condition(geoid_data)].shape[0]
            return choice_count, total_count

        # Calculate required data points
        results = []
        for geoid in filtered_df['geoid'].unique():
            geoid_data = filtered_df[filtered_df['geoid'] == geoid]
            latest_year = geoid_data['year'].max()
            earliest_year = max(latest_year - year_gap, geoid_data['year'].min())
            if earliest_year != latest_year - year_gap:
                continue

            earliest_price = geoid_data[geoid_data['year'] == earliest_year]['price'].median()
            latest_price = geoid_data[geoid_data['year'] == latest_year]['price'].median()
            median_renter_mhi = geoid_data['r_mhi'].median()
            # print(median_renter_mhi, (latest_price - earliest_price))

            unaffordability_index = None    
            # ACCOUNT FOR MHI and Income Growth
            if earliest_price and latest_price and earliest_price != 0:
                if median_renter_mhi > 0:
                    old_income = mdf[mdf['year'] == earliest_year]['MAPCPI'].values[0]
                    new_income = mdf[mdf['year'] == latest_year]['MAPCPI'].values[0]
                    fractional_income_change = 1 + ((new_income - old_income)/old_income)
                    unaffordability_index = ((latest_price - earliest_price)/ (median_renter_mhi*fractional_income_change)) 

            choice_count, total_count = custom_counter(geoid_data)

            if total_count > 0:
                impact_index = ((choice_count / total_count))/global_fraction
                results.append({'geoid': geoid, 'unaffordability_index': unaffordability_index, 'impact_index': impact_index, 'choice_count': choice_count})

        result_df = pd.DataFrame(results)
        result_df = result_df.sort_values(by='unaffordability_index', ascending=True)

        # Filter and drop NaNs
        result_df.dropna(subset=['unaffordability_index', 'impact_index', 'choice_count'], inplace=True)

        # Filter the resulting DataFrame to the middle 95% of the 'price_change_percentage' data
        lower_bound, upper_bound = np.percentile(result_df['unaffordability_index'], [0.5, 95])
        filtered_result_df = result_df[(result_df['unaffordability_index'] >= lower_bound) & (result_df['unaffordability_index'] <= upper_bound)]

        # Using np.histogram to compute the histogram without density
        bins = 50  # You can adjust the number of bins
        bin_edges = np.linspace(filtered_result_df['unaffordability_index'].min(), filtered_result_df['unaffordability_index'].max(), bins+1)
        indices = np.digitize(filtered_result_df['unaffordability_index'], bins=bin_edges)
        bin_averages = [filtered_result_df.loc[indices == i, 'impact_index'].median() for i in range(1, len(bin_edges))]
        bin_counts = [filtered_result_df.loc[indices == i, 'choice_count'].median() for i in range(1, len(bin_edges))]

        # Convert lists to NumPy arrays for plotting
        bin_centers = np.array((bin_edges[:-1] + bin_edges[1:]) / 2)
        bin_averages = np.array(bin_averages)
        bin_counts = np.array(bin_counts)

        # Interpolating NaN values in bin_averages and bin_counts
        valid_avg = ~np.isnan(bin_averages)
        valid_counts = ~np.isnan(bin_counts)

        # If any NaNs exist, perform interpolation
        if np.isnan(bin_averages).any():
            interp_func_avg = interp1d(bin_centers[valid_avg], bin_averages[valid_avg], kind='linear', bounds_error=False, fill_value="extrapolate")
            bin_averages = interp_func_avg(bin_centers)

        if np.isnan(bin_counts).any():
            interp_func_counts = interp1d(bin_centers[valid_counts], bin_counts[valid_counts], kind='linear', bounds_error=False, fill_value="extrapolate")
            bin_counts = interp_func_counts(bin_centers)

        # Create spline fit for smooth curves
        spline_avg = make_interp_spline(bin_centers, bin_averages, k=3)
        spline_count = make_interp_spline(bin_centers, bin_counts, k=3)

        # Generate new x values for a smooth curve
        xnew = np.linspace(bin_centers.min(), bin_centers.max(), 300)

        # Compute the smoothed y values from the spline fits
        y_smooth_avg = spline_avg(xnew)
        y_smooth_count = spline_count(xnew)

        # Prepare plot
        fig = go.Figure()

        fig.add_trace(go.Scatter(showlegend=False, x=xnew, y=y_smooth_avg, line=dict(color='red'), name='%% of your choice impacting property price'))
        fig.add_trace(go.Scatter(x=xnew[xnew < 0], y=y_smooth_avg[xnew < 0], fill='tozeroy', line=dict(color='green'), name='Increasing Affordability'))
        fig.add_trace(go.Scatter(x=xnew[xnew >= 0], y=y_smooth_avg[xnew >= 0], fill='tozeroy', line=dict(color='Red'), name='Increasing Unaffordability'))

        fig.add_trace(go.Scatter(x=xnew, y=y_smooth_count, name='Sales matching <b>your</b> choice', yaxis='y2', line=dict(color='black', dash='dot')))

        # Add median line
        total_area = simps(y_smooth_count, xnew)
        cumulative_area = np.array([simps(y_smooth_count[:i+1], xnew[:i+1]) for i in range(len(xnew))])
        half_total_area = total_area / 2
        index_of_median = np.where(cumulative_area >= half_total_area)[0][0]  # Get the first index where the condition is met
        median_unaffordability_index = xnew[index_of_median]
        fig.add_trace(go.Scatter(showlegend=False, x=[median_unaffordability_index, median_unaffordability_index], 
            y=[0, max(y_smooth_avg.max(), y_smooth_count.max())], mode='lines', 
            line=dict(color='red', dash='dash'), name='Median Unaffordability'))
        # Add annotation for median line
        fig.add_annotation(x=median_unaffordability_index, y=0.9 * y_smooth_avg.max(),  # Adjust y position for text
                        text=f"Median = {median_unaffordability_index:.2f}", showarrow=True,
                        arrowhead=1, ax=60, ay=0) 


        # IMPACT INDEX MEDIAN LINE
        impact_median = np.median(y_smooth_count)
        impact_median_on_y1 = ((y_smooth_avg.max() - y_smooth_avg.min())/(y_smooth_count.max() - y_smooth_count.min())) * impact_median

        fig.add_trace(go.Scatter(showlegend=False, x=[np.min(xnew), np.max(xnew)], 
            y=[impact_median_on_y1, impact_median_on_y1], mode='lines', 
            line=dict(color='red', dash='dash'), name='Median Impact'))
        # Add annotation for median line
        fig.add_annotation(x=xnew.min() + 0.2 * (xnew.max() - xnew.min()), y=impact_median_on_y1 + 0.01 * (y_smooth_avg.max() - y_smooth_avg.min()),  # Adjust y position for text
                        text=f"Median = {impact_median_on_y1:.2f}", showarrow=True,
                        arrowhead=1, ax=0, ay=-30) 


        desired_height = 600  # Set your desired height
        aspect_ratio = (8, 6)  # Set your desired aspect ratio (width:height)

        # Calculate the corresponding width based on the desired height and aspect ratio
        desired_width = int(desired_height * (aspect_ratio[0] / aspect_ratio[1]))

        # Layout adjustments for separate y-axes for counts and percentages
        fig.update_layout(
            title=f'<i>Unaffordability Impact Profile ({year_gap} year period)</i>\
            <br> <b>Choice made: ' + custom_choice_text + '</b>',
            xaxis_title='The Visual Virtuosos <b>Unaffordability Index</b> over all Greater Boston Census Tracts',
            # yaxis_title='% of <b>your choice</b> impacting unaffordability',
            yaxis_title='The Visual Virtuosos <b>Impact Index</b>',
            yaxis_range=[0, max(y_smooth_avg)],
            yaxis2=dict(
                title='Frequency of sales matching <b>your</b> choice',
                overlaying='y',
                side='right',
                range=[0, max(y_smooth_count)]
            ),
            legend=dict(
                orientation="h",
                x=0,
                y=1.02,
                xanchor='left',  # Adjust the horizontal alignment of the legend
                yanchor='bottom'
            ),
            plot_bgcolor='white',
            # legend_title='Legend',
            width=desired_width,  # Set the calculated width
            height=desired_height,  # Set the desired height
            margin=dict(
                t=125,  # Adjust top margin to leave space for the title
                b=50,   # Adjust bottom margin to leave space for the x-axis labels
                l=50,   # Adjust left margin to leave space for the y-axis labels
                r=50    # Adjust right margin to leave space for the legend
            )
        )
        # fig.show()

        plot_json = json.dumps(fig, cls=plotly.utils.PlotlyJSONEncoder)
        with open(f'investor-profiles/public/inv_post/{inv_type}_index.json', 'w') as file:
            file.write(plot_json)
    except Exception as error:
        # handle the exception
        print("An exception occurred:", type(error).__name__, "–", error)        
        print("###################################### FAILED TO GENERATE ######################################")


Small
Medium
Large
Institutional


In [49]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from scipy.interpolate import make_interp_spline, interp1d
from scipy.integrate import simps
from scipy.interpolate import interp1d

# just for this once


for inv_type in inv_color_mapping.keys():
    for budget in [250000, 500000, 750000, 1000000, 1250000, 1500000]:
        print(inv_type, budget)
        try:

            # CUSTOM DFINITION
            def choice_condition(x):
                return ((x['investor_type_purchase'] == inv_type) &
                    (x['price'] <= budget))

            def total_condition(x):
                return ((x['investor_type_purchase'] == inv_type))

            year_gap = 5
            custom_choice_text = f'${budget//1000}k Budget'

            # Assuming df is your DataFrame after the join and has been loaded correctly
            # Filter the resulting DataFrame to the middle 95% of the 'price_change_percentage' data
            lower_bound, upper_bound = np.percentile(df['price'], [2.5, 97.5])
            filtered_df = df[(df['price'] >= lower_bound) & (df['price'] <= upper_bound)]
            global_fraction = filtered_df[choice_condition(filtered_df)].shape[0] / len(filtered_df)

            def custom_counter(geoid_data):
                choice_count = geoid_data[choice_condition(geoid_data)].shape[0]
                total_count = geoid_data[total_condition(geoid_data)].shape[0]
                return choice_count, total_count

            # Calculate required data points
            results = []
            for geoid in filtered_df['geoid'].unique():
                geoid_data = filtered_df[filtered_df['geoid'] == geoid]
                latest_year = geoid_data['year'].max()
                earliest_year = max(latest_year - year_gap, geoid_data['year'].min())
                if earliest_year != latest_year - year_gap:
                    continue

                earliest_price = geoid_data[geoid_data['year'] == earliest_year]['price'].median()
                latest_price = geoid_data[geoid_data['year'] == latest_year]['price'].median()
                median_renter_mhi = geoid_data['r_mhi'].median()
                # print(median_renter_mhi, (latest_price - earliest_price))

                unaffordability_index = None    
                # ACCOUNT FOR MHI and Income Growth
                if earliest_price and latest_price and earliest_price != 0:
                    if median_renter_mhi > 0:
                        old_income = mdf[mdf['year'] == earliest_year]['MAPCPI'].values[0]
                        new_income = mdf[mdf['year'] == latest_year]['MAPCPI'].values[0]
                        fractional_income_change = 1 + ((new_income - old_income)/old_income)
                        unaffordability_index = ((latest_price - earliest_price)/ (median_renter_mhi*fractional_income_change)) 

                choice_count, total_count = custom_counter(geoid_data)

                if total_count > 0:
                    impact_index = ((choice_count / total_count))/global_fraction
                    results.append({'geoid': geoid, 'unaffordability_index': unaffordability_index, 'impact_index': impact_index, 'choice_count': choice_count})

            result_df = pd.DataFrame(results)
            result_df = result_df.sort_values(by='unaffordability_index', ascending=True)

            # Filter and drop NaNs
            result_df.dropna(subset=['unaffordability_index', 'impact_index', 'choice_count'], inplace=True)

            # Filter the resulting DataFrame to the middle 95% of the 'price_change_percentage' data
            lower_bound, upper_bound = np.percentile(result_df['unaffordability_index'], [0.5, 95])
            filtered_result_df = result_df[(result_df['unaffordability_index'] >= lower_bound) & (result_df['unaffordability_index'] <= upper_bound)]

            # Using np.histogram to compute the histogram without density
            bins = 50  # You can adjust the number of bins
            bin_edges = np.linspace(filtered_result_df['unaffordability_index'].min(), filtered_result_df['unaffordability_index'].max(), bins+1)
            indices = np.digitize(filtered_result_df['unaffordability_index'], bins=bin_edges)
            bin_averages = [filtered_result_df.loc[indices == i, 'impact_index'].median() for i in range(1, len(bin_edges))]
            bin_counts = [filtered_result_df.loc[indices == i, 'choice_count'].median() for i in range(1, len(bin_edges))]

            # Convert lists to NumPy arrays for plotting
            bin_centers = np.array((bin_edges[:-1] + bin_edges[1:]) / 2)
            bin_averages = np.array(bin_averages)
            bin_counts = np.array(bin_counts)

            # Interpolating NaN values in bin_averages and bin_counts
            valid_avg = ~np.isnan(bin_averages)
            valid_counts = ~np.isnan(bin_counts)

            # If any NaNs exist, perform interpolation
            if np.isnan(bin_averages).any():
                interp_func_avg = interp1d(bin_centers[valid_avg], bin_averages[valid_avg], kind='linear', bounds_error=False, fill_value="extrapolate")
                bin_averages = interp_func_avg(bin_centers)

            if np.isnan(bin_counts).any():
                interp_func_counts = interp1d(bin_centers[valid_counts], bin_counts[valid_counts], kind='linear', bounds_error=False, fill_value="extrapolate")
                bin_counts = interp_func_counts(bin_centers)

            # Create spline fit for smooth curves
            spline_avg = make_interp_spline(bin_centers, bin_averages, k=3)
            spline_count = make_interp_spline(bin_centers, bin_counts, k=3)

            # Generate new x values for a smooth curve
            xnew = np.linspace(bin_centers.min(), bin_centers.max(), 300)

            # Compute the smoothed y values from the spline fits
            y_smooth_avg = spline_avg(xnew)
            y_smooth_count = spline_count(xnew)

            # Prepare plot
            fig = go.Figure()

            fig.add_trace(go.Scatter(showlegend=False, x=xnew, y=y_smooth_avg, line=dict(color='red'), name='%% of your choice impacting property price'))
            fig.add_trace(go.Scatter(x=xnew[xnew < 0], y=y_smooth_avg[xnew < 0], fill='tozeroy', line=dict(color='green'), name='Increasing Affordability'))
            fig.add_trace(go.Scatter(x=xnew[xnew >= 0], y=y_smooth_avg[xnew >= 0], fill='tozeroy', line=dict(color='Red'), name='Increasing Unaffordability'))

            fig.add_trace(go.Scatter(x=xnew, y=y_smooth_count, name='Sales matching <b>your</b> choice', yaxis='y2', line=dict(color='black', dash='dot')))

            # Add median line
            total_area = simps(y_smooth_count, xnew)
            cumulative_area = np.array([simps(y_smooth_count[:i+1], xnew[:i+1]) for i in range(len(xnew))])
            half_total_area = total_area / 2
            index_of_median = np.where(cumulative_area >= half_total_area)[0][0]  # Get the first index where the condition is met
            median_unaffordability_index = xnew[index_of_median]
            fig.add_trace(go.Scatter(showlegend=False, x=[median_unaffordability_index, median_unaffordability_index], 
                y=[0, max(y_smooth_avg.max(), y_smooth_count.max())], mode='lines', 
                line=dict(color='red', dash='dash'), name='Median Unaffordability'))
            # Add annotation for median line
            fig.add_annotation(x=median_unaffordability_index, y=0.9 * y_smooth_avg.max(),  # Adjust y position for text
                            text=f"Median = {median_unaffordability_index:.2f}", showarrow=True,
                            arrowhead=1, ax=60, ay=0) 


            # IMPACT INDEX MEDIAN LINE
            impact_median = np.median(y_smooth_count)
            impact_median_on_y1 = ((y_smooth_avg.max() - y_smooth_avg.min())/(y_smooth_count.max() - y_smooth_count.min())) * impact_median

            fig.add_trace(go.Scatter(showlegend=False, x=[np.min(xnew), np.max(xnew)], 
                y=[impact_median_on_y1, impact_median_on_y1], mode='lines', 
                line=dict(color='red', dash='dash'), name='Median Impact'))
            # Add annotation for median line
            fig.add_annotation(x=xnew.min() + 0.2 * (xnew.max() - xnew.min()), y=impact_median_on_y1 + 0.01 * (y_smooth_avg.max() - y_smooth_avg.min()),  # Adjust y position for text
                            text=f"Median = {impact_median_on_y1:.2f}", showarrow=True,
                            arrowhead=1, ax=0, ay=-30) 


            desired_height = 600  # Set your desired height
            aspect_ratio = (8, 6)  # Set your desired aspect ratio (width:height)

            # Calculate the corresponding width based on the desired height and aspect ratio
            desired_width = int(desired_height * (aspect_ratio[0] / aspect_ratio[1]))

            # Layout adjustments for separate y-axes for counts and percentages
            fig.update_layout(
                title=f'<i>Unaffordability Impact Profile ({year_gap} year period)</i>\
                <br> <b>Choice made: ' + custom_choice_text + '</b>',
                xaxis_title='The Visual Virtuosos <b>Unaffordability Index</b> over all Greater Boston Census Tracts',
                # yaxis_title='% of <b>your choice</b> impacting unaffordability',
                yaxis_title='The Visual Virtuosos <b>Impact Index</b>',
                yaxis_range=[0, max(y_smooth_avg)],
                yaxis2=dict(
                    title='Frequency of sales matching <b>your</b> choice',
                    overlaying='y',
                    side='right',
                    range=[0, max(y_smooth_count)]
                ),
                legend=dict(
                    orientation="h",
                    x=0,
                    y=1.02,
                    xanchor='left',  # Adjust the horizontal alignment of the legend
                    yanchor='bottom'
                ),
                plot_bgcolor='white',
                # legend_title='Legend',
                width=desired_width,  # Set the calculated width
                height=desired_height,  # Set the desired height
                margin=dict(
                    t=125,  # Adjust top margin to leave space for the title
                    b=50,   # Adjust bottom margin to leave space for the x-axis labels
                    l=50,   # Adjust left margin to leave space for the y-axis labels
                    r=50    # Adjust right margin to leave space for the legend
                )
            )
            # fig.show()

            plot_json = json.dumps(fig, cls=plotly.utils.PlotlyJSONEncoder)
            with open(f'investor-profiles/public/budget_post/{inv_type}_{budget}_index.json', 'w') as file:
                file.write(plot_json)
        except Exception as error:
            # handle the exception
            print("An exception occurred:", type(error).__name__, "–", error)        
            print("###################################### FAILED TO GENERATE ######################################")


Small 250000
Small 500000
Small 750000
Small 1000000
Small 1250000
Small 1500000
Medium 250000
Medium 500000
Medium 750000
Medium 1000000
Medium 1250000
Medium 1500000
Large 250000
Large 500000
Large 750000
Large 1000000
Large 1250000
Large 1500000
Institutional 250000
Institutional 500000
Institutional 750000
Institutional 1000000
Institutional 1250000
Institutional 1500000


In [50]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from scipy.interpolate import make_interp_spline, interp1d
from scipy.integrate import simps
from scipy.interpolate import interp1d

# just for this once


for inv_type in inv_color_mapping.keys():
    for budget in [250000, 500000, 750000, 1000000, 1250000, 1500000]:
        for style in ['Traditional', 'Modern', 'Multi-Unit']:
            print(inv_type, budget, style)
            try:

                # CUSTOM DFINITION
                def choice_condition(x):
                    return ((x['investor_type_purchase'] == inv_type) &
                        (x['price'] <= budget) &
                        (x['style_category'] == style))

                def total_condition(x):
                    return ((x['investor_type_purchase'] == inv_type) &
                        (x['price'] <= budget))

                year_gap = 5
                custom_choice_text = f'{style} Property Style'

                # Assuming df is your DataFrame after the join and has been loaded correctly
                # Filter the resulting DataFrame to the middle 95% of the 'price_change_percentage' data
                lower_bound, upper_bound = np.percentile(df['price'], [2.5, 97.5])
                filtered_df = df[(df['price'] >= lower_bound) & (df['price'] <= upper_bound)]
                global_fraction = filtered_df[choice_condition(filtered_df)].shape[0] / len(filtered_df)

                def custom_counter(geoid_data):
                    choice_count = geoid_data[choice_condition(geoid_data)].shape[0]
                    total_count = geoid_data[total_condition(geoid_data)].shape[0]
                    return choice_count, total_count


                # Calculate required data points
                results = []
                for geoid in filtered_df['geoid'].unique():
                    geoid_data = filtered_df[filtered_df['geoid'] == geoid]
                    latest_year = geoid_data['year'].max()
                    earliest_year = max(latest_year - year_gap, geoid_data['year'].min())
                    if earliest_year != latest_year - year_gap:
                        continue

                    earliest_price = geoid_data[geoid_data['year'] == earliest_year]['price'].median()
                    latest_price = geoid_data[geoid_data['year'] == latest_year]['price'].median()
                    median_renter_mhi = geoid_data['r_mhi'].median()
                    # print(median_renter_mhi, (latest_price - earliest_price))

                    unaffordability_index = None    
                    # ACCOUNT FOR MHI and Income Growth
                    if earliest_price and latest_price and earliest_price != 0:
                        if median_renter_mhi > 0:
                            old_income = mdf[mdf['year'] == earliest_year]['MAPCPI'].values[0]
                            new_income = mdf[mdf['year'] == latest_year]['MAPCPI'].values[0]
                            fractional_income_change = 1 + ((new_income - old_income)/old_income)
                            unaffordability_index = ((latest_price - earliest_price)/ (median_renter_mhi*fractional_income_change)) 

                    choice_count, total_count = custom_counter(geoid_data)

                    if total_count > 0:
                        impact_index = ((choice_count / total_count))/global_fraction
                        results.append({'geoid': geoid, 'unaffordability_index': unaffordability_index, 'impact_index': impact_index, 'choice_count': choice_count})

                result_df = pd.DataFrame(results)
                result_df = result_df.sort_values(by='unaffordability_index', ascending=True)

                # Filter and drop NaNs
                result_df.dropna(subset=['unaffordability_index', 'impact_index', 'choice_count'], inplace=True)

                # Filter the resulting DataFrame to the middle 95% of the 'price_change_percentage' data
                lower_bound, upper_bound = np.percentile(result_df['unaffordability_index'], [0.5, 95])
                filtered_result_df = result_df[(result_df['unaffordability_index'] >= lower_bound) & (result_df['unaffordability_index'] <= upper_bound)]

                # Using np.histogram to compute the histogram without density
                bins = 50  # You can adjust the number of bins
                bin_edges = np.linspace(filtered_result_df['unaffordability_index'].min(), filtered_result_df['unaffordability_index'].max(), bins+1)
                indices = np.digitize(filtered_result_df['unaffordability_index'], bins=bin_edges)
                bin_averages = [filtered_result_df.loc[indices == i, 'impact_index'].median() for i in range(1, len(bin_edges))]
                bin_counts = [filtered_result_df.loc[indices == i, 'choice_count'].median() for i in range(1, len(bin_edges))]

                # Convert lists to NumPy arrays for plotting
                bin_centers = np.array((bin_edges[:-1] + bin_edges[1:]) / 2)
                bin_averages = np.array(bin_averages)
                bin_counts = np.array(bin_counts)

                # Interpolating NaN values in bin_averages and bin_counts
                valid_avg = ~np.isnan(bin_averages)
                valid_counts = ~np.isnan(bin_counts)

                # If any NaNs exist, perform interpolation
                if np.isnan(bin_averages).any():
                    interp_func_avg = interp1d(bin_centers[valid_avg], bin_averages[valid_avg], kind='linear', bounds_error=False, fill_value="extrapolate")
                    bin_averages = interp_func_avg(bin_centers)

                if np.isnan(bin_counts).any():
                    interp_func_counts = interp1d(bin_centers[valid_counts], bin_counts[valid_counts], kind='linear', bounds_error=False, fill_value="extrapolate")
                    bin_counts = interp_func_counts(bin_centers)

                # Create spline fit for smooth curves
                spline_avg = make_interp_spline(bin_centers, bin_averages, k=3)
                spline_count = make_interp_spline(bin_centers, bin_counts, k=3)

                # Generate new x values for a smooth curve
                xnew = np.linspace(bin_centers.min(), bin_centers.max(), 300)

                # Compute the smoothed y values from the spline fits
                y_smooth_avg = spline_avg(xnew)
                y_smooth_count = spline_count(xnew)

                # Prepare plot
                fig = go.Figure()

                fig.add_trace(go.Scatter(showlegend=False, x=xnew, y=y_smooth_avg, line=dict(color='red'), name='%% of your choice impacting property price'))
                fig.add_trace(go.Scatter(x=xnew[xnew < 0], y=y_smooth_avg[xnew < 0], fill='tozeroy', line=dict(color='green'), name='Increasing Affordability'))
                fig.add_trace(go.Scatter(x=xnew[xnew >= 0], y=y_smooth_avg[xnew >= 0], fill='tozeroy', line=dict(color='Red'), name='Increasing Unaffordability'))

                fig.add_trace(go.Scatter(x=xnew, y=y_smooth_count, name='Sales matching <b>your</b> choice', yaxis='y2', line=dict(color='black', dash='dot')))

                # Add median line
                total_area = simps(y_smooth_count, xnew)
                cumulative_area = np.array([simps(y_smooth_count[:i+1], xnew[:i+1]) for i in range(len(xnew))])
                half_total_area = total_area / 2
                index_of_median = np.where(cumulative_area >= half_total_area)[0][0]  # Get the first index where the condition is met
                median_unaffordability_index = xnew[index_of_median]
                fig.add_trace(go.Scatter(showlegend=False, x=[median_unaffordability_index, median_unaffordability_index], 
                    y=[0, max(y_smooth_avg.max(), y_smooth_count.max())], mode='lines', 
                    line=dict(color='red', dash='dash'), name='Median Unaffordability'))
                # Add annotation for median line
                fig.add_annotation(x=median_unaffordability_index, y=0.9 * y_smooth_avg.max(),  # Adjust y position for text
                                text=f"Median = {median_unaffordability_index:.2f}", showarrow=True,
                                arrowhead=1, ax=60, ay=0) 


                # IMPACT INDEX MEDIAN LINE
                impact_median = np.median(y_smooth_count)
                impact_median_on_y1 = ((y_smooth_avg.max() - y_smooth_avg.min())/(y_smooth_count.max() - y_smooth_count.min())) * impact_median

                fig.add_trace(go.Scatter(showlegend=False, x=[np.min(xnew), np.max(xnew)], 
                    y=[impact_median_on_y1, impact_median_on_y1], mode='lines', 
                    line=dict(color='red', dash='dash'), name='Median Impact'))
                # Add annotation for median line
                fig.add_annotation(x=xnew.min() + 0.2 * (xnew.max() - xnew.min()), y=impact_median_on_y1 + 0.01 * (y_smooth_avg.max() - y_smooth_avg.min()),  # Adjust y position for text
                                text=f"Median = {impact_median_on_y1:.2f}", showarrow=True,
                                arrowhead=1, ax=0, ay=-30) 


                desired_height = 600  # Set your desired height
                aspect_ratio = (8, 6)  # Set your desired aspect ratio (width:height)

                # Calculate the corresponding width based on the desired height and aspect ratio
                desired_width = int(desired_height * (aspect_ratio[0] / aspect_ratio[1]))

                # Layout adjustments for separate y-axes for counts and percentages
                fig.update_layout(
                    title=f'<i>Unaffordability Impact Profile ({year_gap} year period)</i>\
                    <br> <b>Choice made: ' + custom_choice_text + '</b>',
                    xaxis_title='The Visual Virtuosos <b>Unaffordability Index</b> over all Greater Boston Census Tracts',
                    # yaxis_title='% of <b>your choice</b> impacting unaffordability',
                    yaxis_title='The Visual Virtuosos <b>Impact Index</b>',
                    yaxis_range=[0, max(y_smooth_avg)],
                    yaxis2=dict(
                        title='Frequency of sales matching <b>your</b> choice',
                        overlaying='y',
                        side='right',
                        range=[0, max(y_smooth_count)]
                    ),
                    legend=dict(
                        orientation="h",
                        x=0,
                        y=1.02,
                        xanchor='left',  # Adjust the horizontal alignment of the legend
                        yanchor='bottom'
                    ),
                    plot_bgcolor='white',
                    # legend_title='Legend',
                    width=desired_width,  # Set the calculated width
                    height=desired_height,  # Set the desired height
                    margin=dict(
                        t=125,  # Adjust top margin to leave space for the title
                        b=50,   # Adjust bottom margin to leave space for the x-axis labels
                        l=50,   # Adjust left margin to leave space for the y-axis labels
                        r=50    # Adjust right margin to leave space for the legend
                    )
                )
                # fig.show()

                plot_json = json.dumps(fig, cls=plotly.utils.PlotlyJSONEncoder)
                with open(f'investor-profiles/public/style_post/{inv_type}_{budget}_{style}_index.json', 'w') as file:
                    file.write(plot_json)
            except Exception as error:
                # handle the exception
                print("An exception occurred:", type(error).__name__, "–", error)        
                print("###################################### FAILED TO GENERATE ######################################")


Small 250000 Traditional
Small 250000 Modern
Small 250000 Multi-Unit
Small 500000 Traditional
Small 500000 Modern
Small 500000 Multi-Unit
Small 750000 Traditional
Small 750000 Modern
Small 750000 Multi-Unit
Small 1000000 Traditional
Small 1000000 Modern
Small 1000000 Multi-Unit
Small 1250000 Traditional
Small 1250000 Modern
Small 1250000 Multi-Unit
Small 1500000 Traditional
Small 1500000 Modern
Small 1500000 Multi-Unit
Medium 250000 Traditional
Medium 250000 Modern
Medium 250000 Multi-Unit
Medium 500000 Traditional
Medium 500000 Modern
Medium 500000 Multi-Unit
Medium 750000 Traditional
Medium 750000 Modern
Medium 750000 Multi-Unit
Medium 1000000 Traditional
Medium 1000000 Modern
Medium 1000000 Multi-Unit
Medium 1250000 Traditional
Medium 1250000 Modern
Medium 1250000 Multi-Unit
Medium 1500000 Traditional
Medium 1500000 Modern
Medium 1500000 Multi-Unit
Large 250000 Traditional
Large 250000 Modern
Large 250000 Multi-Unit
Large 500000 Traditional
Large 500000 Modern
Large 500000 Multi-Uni

In [52]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from scipy.interpolate import make_interp_spline, interp1d
from scipy.integrate import simps
from scipy.interpolate import interp1d

# just for this once


for inv_type in inv_color_mapping.keys():
    for budget in [250000, 500000, 750000, 1000000, 1250000, 1500000]:
        for style in ['Traditional', 'Modern', 'Multi-Unit']:
            for subregion in list(df['subregion'].unique()):
                print(inv_type, budget, style, subregion)
                try:

                    # CUSTOM DFINITION
                    def choice_condition(x):
                        return ((x['investor_type_purchase'] == inv_type) &
                            (x['price'] <= budget) &
                            (x['style_category'] == style) &
                            (x['subregion'] == subregion))

                    def total_condition(x):
                        return ((x['investor_type_purchase'] == inv_type) &
                            (x['price'] <= budget) &
                            (x['style_category'] == style))

                    year_gap = 5
                    custom_choice_text = f'{subregion} MAPC Sub-region'

                    # Assuming df is your DataFrame after the join and has been loaded correctly
                    # Filter the resulting DataFrame to the middle 95% of the 'price_change_percentage' data
                    lower_bound, upper_bound = np.percentile(df['price'], [2.5, 97.5])
                    filtered_df = df[(df['price'] >= lower_bound) & (df['price'] <= upper_bound)]
                    global_fraction = filtered_df[choice_condition(filtered_df)].shape[0] / len(filtered_df)

                    def custom_counter(geoid_data):
                        choice_count = geoid_data[choice_condition(geoid_data)].shape[0]
                        total_count = geoid_data[total_condition(geoid_data)].shape[0]
                        return choice_count, total_count


                    # Calculate required data points
                    results = []
                    for geoid in filtered_df['geoid'].unique():
                        geoid_data = filtered_df[filtered_df['geoid'] == geoid]
                        latest_year = geoid_data['year'].max()
                        earliest_year = max(latest_year - year_gap, geoid_data['year'].min())
                        if earliest_year != latest_year - year_gap:
                            continue

                        earliest_price = geoid_data[geoid_data['year'] == earliest_year]['price'].median()
                        latest_price = geoid_data[geoid_data['year'] == latest_year]['price'].median()
                        median_renter_mhi = geoid_data['r_mhi'].median()
                        # print(median_renter_mhi, (latest_price - earliest_price))

                        unaffordability_index = None    
                        # ACCOUNT FOR MHI and Income Growth
                        if earliest_price and latest_price and earliest_price != 0:
                            if median_renter_mhi > 0:
                                old_income = mdf[mdf['year'] == earliest_year]['MAPCPI'].values[0]
                                new_income = mdf[mdf['year'] == latest_year]['MAPCPI'].values[0]
                                fractional_income_change = 1 + ((new_income - old_income)/old_income)
                                unaffordability_index = ((latest_price - earliest_price)/ (median_renter_mhi*fractional_income_change)) 

                        choice_count, total_count = custom_counter(geoid_data)

                        if total_count > 0:
                            impact_index = ((choice_count / total_count))/global_fraction
                            results.append({'geoid': geoid, 'unaffordability_index': unaffordability_index, 'impact_index': impact_index, 'choice_count': choice_count})

                    result_df = pd.DataFrame(results)
                    result_df = result_df.sort_values(by='unaffordability_index', ascending=True)

                    # Filter and drop NaNs
                    result_df.dropna(subset=['unaffordability_index', 'impact_index', 'choice_count'], inplace=True)

                    # Filter the resulting DataFrame to the middle 95% of the 'price_change_percentage' data
                    lower_bound, upper_bound = np.percentile(result_df['unaffordability_index'], [0.5, 95])
                    filtered_result_df = result_df[(result_df['unaffordability_index'] >= lower_bound) & (result_df['unaffordability_index'] <= upper_bound)]

                    # Using np.histogram to compute the histogram without density
                    bins = 50  # You can adjust the number of bins
                    bin_edges = np.linspace(filtered_result_df['unaffordability_index'].min(), filtered_result_df['unaffordability_index'].max(), bins+1)
                    indices = np.digitize(filtered_result_df['unaffordability_index'], bins=bin_edges)
                    bin_averages = [filtered_result_df.loc[indices == i, 'impact_index'].median() for i in range(1, len(bin_edges))]
                    bin_counts = [filtered_result_df.loc[indices == i, 'choice_count'].median() for i in range(1, len(bin_edges))]

                    # Convert lists to NumPy arrays for plotting
                    bin_centers = np.array((bin_edges[:-1] + bin_edges[1:]) / 2)
                    bin_averages = np.array(bin_averages)
                    bin_counts = np.array(bin_counts)

                    # Interpolating NaN values in bin_averages and bin_counts
                    valid_avg = ~np.isnan(bin_averages)
                    valid_counts = ~np.isnan(bin_counts)

                    # If any NaNs exist, perform interpolation
                    if np.isnan(bin_averages).any():
                        interp_func_avg = interp1d(bin_centers[valid_avg], bin_averages[valid_avg], kind='linear', bounds_error=False, fill_value="extrapolate")
                        bin_averages = interp_func_avg(bin_centers)

                    if np.isnan(bin_counts).any():
                        interp_func_counts = interp1d(bin_centers[valid_counts], bin_counts[valid_counts], kind='linear', bounds_error=False, fill_value="extrapolate")
                        bin_counts = interp_func_counts(bin_centers)

                    # Create spline fit for smooth curves
                    spline_avg = make_interp_spline(bin_centers, bin_averages, k=3)
                    spline_count = make_interp_spline(bin_centers, bin_counts, k=3)

                    # Generate new x values for a smooth curve
                    xnew = np.linspace(bin_centers.min(), bin_centers.max(), 300)

                    # Compute the smoothed y values from the spline fits
                    y_smooth_avg = spline_avg(xnew)
                    y_smooth_count = spline_count(xnew)

                    # Prepare plot
                    fig = go.Figure()

                    fig.add_trace(go.Scatter(showlegend=False, x=xnew, y=y_smooth_avg, line=dict(color='red'), name='%% of your choice impacting property price'))
                    fig.add_trace(go.Scatter(x=xnew[xnew < 0], y=y_smooth_avg[xnew < 0], fill='tozeroy', line=dict(color='green'), name='Increasing Affordability'))
                    fig.add_trace(go.Scatter(x=xnew[xnew >= 0], y=y_smooth_avg[xnew >= 0], fill='tozeroy', line=dict(color='Red'), name='Increasing Unaffordability'))

                    fig.add_trace(go.Scatter(x=xnew, y=y_smooth_count, name='Sales matching <b>your</b> choice', yaxis='y2', line=dict(color='black', dash='dot')))

                    # Add median line
                    total_area = simps(y_smooth_count, xnew)
                    cumulative_area = np.array([simps(y_smooth_count[:i+1], xnew[:i+1]) for i in range(len(xnew))])
                    half_total_area = total_area / 2
                    index_of_median = np.where(cumulative_area >= half_total_area)[0][0]  # Get the first index where the condition is met
                    median_unaffordability_index = xnew[index_of_median]
                    fig.add_trace(go.Scatter(showlegend=False, x=[median_unaffordability_index, median_unaffordability_index], 
                        y=[0, max(y_smooth_avg.max(), y_smooth_count.max())], mode='lines', 
                        line=dict(color='red', dash='dash'), name='Median Unaffordability'))
                    # Add annotation for median line
                    fig.add_annotation(x=median_unaffordability_index, y=0.9 * y_smooth_avg.max(),  # Adjust y position for text
                                    text=f"Median = {median_unaffordability_index:.2f}", showarrow=True,
                                    arrowhead=1, ax=60, ay=0) 


                    # IMPACT INDEX MEDIAN LINE
                    impact_median = np.median(y_smooth_count)
                    impact_median_on_y1 = ((y_smooth_avg.max() - y_smooth_avg.min())/(y_smooth_count.max() - y_smooth_count.min())) * impact_median

                    fig.add_trace(go.Scatter(showlegend=False, x=[np.min(xnew), np.max(xnew)], 
                        y=[impact_median_on_y1, impact_median_on_y1], mode='lines', 
                        line=dict(color='red', dash='dash'), name='Median Impact'))
                    # Add annotation for median line
                    fig.add_annotation(x=xnew.min() + 0.2 * (xnew.max() - xnew.min()), y=impact_median_on_y1 + 0.01 * (y_smooth_avg.max() - y_smooth_avg.min()),  # Adjust y position for text
                                    text=f"Median = {impact_median_on_y1:.2f}", showarrow=True,
                                    arrowhead=1, ax=0, ay=-30) 


                    desired_height = 600  # Set your desired height
                    aspect_ratio = (8, 6)  # Set your desired aspect ratio (width:height)

                    # Calculate the corresponding width based on the desired height and aspect ratio
                    desired_width = int(desired_height * (aspect_ratio[0] / aspect_ratio[1]))

                    # Layout adjustments for separate y-axes for counts and percentages
                    fig.update_layout(
                        title=f'<i>Unaffordability Impact Profile ({year_gap} year period)</i>\
                        <br> <b>Choice made: ' + custom_choice_text + '</b>',
                        xaxis_title='The Visual Virtuosos <b>Unaffordability Index</b> over all Greater Boston Census Tracts',
                        # yaxis_title='% of <b>your choice</b> impacting unaffordability',
                        yaxis_title='The Visual Virtuosos <b>Impact Index</b>',
                        yaxis_range=[0, max(y_smooth_avg)],
                        yaxis2=dict(
                            title='Frequency of sales matching <b>your</b> choice',
                            overlaying='y',
                            side='right',
                            range=[0, max(y_smooth_count)]
                        ),
                        legend=dict(
                            orientation="h",
                            x=0,
                            y=1.02,
                            xanchor='left',  # Adjust the horizontal alignment of the legend
                            yanchor='bottom'
                        ),
                        plot_bgcolor='white',
                        # legend_title='Legend',
                        width=desired_width,  # Set the calculated width
                        height=desired_height,  # Set the desired height
                        margin=dict(
                            t=125,  # Adjust top margin to leave space for the title
                            b=50,   # Adjust bottom margin to leave space for the x-axis labels
                            l=50,   # Adjust left margin to leave space for the y-axis labels
                            r=50    # Adjust right margin to leave space for the legend
                        )
                    )
                    # fig.show()

                    plot_json = json.dumps(fig, cls=plotly.utils.PlotlyJSONEncoder)
                    with open(f'investor-profiles/public/loc_post/{inv_type}_{budget}_{style}_{subregion}_index.json', 'w') as file:
                        file.write(plot_json)
                except Exception as error:
                    # handle the exception
                    print("An exception occurred:", type(error).__name__, "–", error)        
                    print("###################################### FAILED TO GENERATE ######################################")


Small 250000 Traditional SSC
Small 250000 Traditional ICC
Small 250000 Traditional MAGIC
Small 250000 Traditional NSPC
Small 250000 Traditional NSTF
Small 250000 Traditional MWRC
Small 250000 Traditional SWAP
Small 250000 Traditional TRIC
Small 250000 Modern SSC
Small 250000 Modern ICC
Small 250000 Modern MAGIC
Small 250000 Modern NSPC
Small 250000 Modern NSTF
Small 250000 Modern MWRC
Small 250000 Modern SWAP
Small 250000 Modern TRIC
Small 250000 Multi-Unit SSC
Small 250000 Multi-Unit ICC
Small 250000 Multi-Unit MAGIC
Small 250000 Multi-Unit NSPC
Small 250000 Multi-Unit NSTF
Small 250000 Multi-Unit MWRC
Small 250000 Multi-Unit SWAP
Small 250000 Multi-Unit TRIC
Small 500000 Traditional SSC
Small 500000 Traditional ICC
Small 500000 Traditional MAGIC
Small 500000 Traditional NSPC
Small 500000 Traditional NSTF
Small 500000 Traditional MWRC
Small 500000 Traditional SWAP
Small 500000 Traditional TRIC
Small 500000 Modern SSC
Small 500000 Modern ICC
Small 500000 Modern MAGIC
Small 500000 Moder

In [53]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from scipy.interpolate import make_interp_spline, interp1d
from scipy.integrate import simps
from scipy.interpolate import interp1d

# just for this once


for inv_type in inv_color_mapping.keys():
    for budget in [250000, 500000, 750000, 1000000, 1250000, 1500000]:
        for style in ['Traditional', 'Modern', 'Multi-Unit']:
            for subregion in list(df['subregion'].unique()):
                for flip_var in ['NoFlip', 'Flip']:
                    print(inv_type, budget, style, subregion, flip_var)
                    try:

                        # CUSTOM DFINITION
                        def choice_condition(x):
                            return ((x['investor_type_purchase'] == inv_type) &
                                (x['price'] <= budget) &
                                (x['style_category'] == style) &
                                (x['subregion'] == subregion) &        
                                (x['flip_status'] == flip_var))

                        def total_condition(x):
                            return ((x['investor_type_purchase'] == inv_type) &
                                (x['price'] <= budget) &
                                (x['style_category'] == style) &
                                (x['subregion'] == subregion) &        
                                (x['flip_status'].notna()))

                        year_gap = 5
                        custom_choice_text = f'{flip_var} strategy'

                        # Assuming df is your DataFrame after the join and has been loaded correctly
                        # Filter the resulting DataFrame to the middle 95% of the 'price_change_percentage' data
                        lower_bound, upper_bound = np.percentile(df['price'], [2.5, 97.5])
                        filtered_df = df[(df['price'] >= lower_bound) & (df['price'] <= upper_bound)]
                        global_fraction = filtered_df[choice_condition(filtered_df)].shape[0] / len(filtered_df)

                        def custom_counter(geoid_data):
                            choice_count = geoid_data[choice_condition(geoid_data)].shape[0]
                            total_count = geoid_data[total_condition(geoid_data)].shape[0]
                            return choice_count, total_count


                        # Calculate required data points
                        results = []
                        for geoid in filtered_df['geoid'].unique():
                            geoid_data = filtered_df[filtered_df['geoid'] == geoid]
                            latest_year = geoid_data['year'].max()
                            earliest_year = max(latest_year - year_gap, geoid_data['year'].min())
                            if earliest_year != latest_year - year_gap:
                                continue

                            earliest_price = geoid_data[geoid_data['year'] == earliest_year]['price'].median()
                            latest_price = geoid_data[geoid_data['year'] == latest_year]['price'].median()
                            median_renter_mhi = geoid_data['r_mhi'].median()
                            # print(median_renter_mhi, (latest_price - earliest_price))

                            unaffordability_index = None    
                            # ACCOUNT FOR MHI and Income Growth
                            if earliest_price and latest_price and earliest_price != 0:
                                if median_renter_mhi > 0:
                                    old_income = mdf[mdf['year'] == earliest_year]['MAPCPI'].values[0]
                                    new_income = mdf[mdf['year'] == latest_year]['MAPCPI'].values[0]
                                    fractional_income_change = 1 + ((new_income - old_income)/old_income)
                                    unaffordability_index = ((latest_price - earliest_price)/ (median_renter_mhi*fractional_income_change)) 

                            choice_count, total_count = custom_counter(geoid_data)

                            if total_count > 0:
                                impact_index = ((choice_count / total_count))/global_fraction
                                results.append({'geoid': geoid, 'unaffordability_index': unaffordability_index, 'impact_index': impact_index, 'choice_count': choice_count})

                        result_df = pd.DataFrame(results)
                        result_df = result_df.sort_values(by='unaffordability_index', ascending=True)

                        # Filter and drop NaNs
                        result_df.dropna(subset=['unaffordability_index', 'impact_index', 'choice_count'], inplace=True)

                        # Filter the resulting DataFrame to the middle 95% of the 'price_change_percentage' data
                        lower_bound, upper_bound = np.percentile(result_df['unaffordability_index'], [0.5, 95])
                        filtered_result_df = result_df[(result_df['unaffordability_index'] >= lower_bound) & (result_df['unaffordability_index'] <= upper_bound)]

                        # Using np.histogram to compute the histogram without density
                        bins = 50  # You can adjust the number of bins
                        bin_edges = np.linspace(filtered_result_df['unaffordability_index'].min(), filtered_result_df['unaffordability_index'].max(), bins+1)
                        indices = np.digitize(filtered_result_df['unaffordability_index'], bins=bin_edges)
                        bin_averages = [filtered_result_df.loc[indices == i, 'impact_index'].median() for i in range(1, len(bin_edges))]
                        bin_counts = [filtered_result_df.loc[indices == i, 'choice_count'].median() for i in range(1, len(bin_edges))]

                        # Convert lists to NumPy arrays for plotting
                        bin_centers = np.array((bin_edges[:-1] + bin_edges[1:]) / 2)
                        bin_averages = np.array(bin_averages)
                        bin_counts = np.array(bin_counts)

                        # Interpolating NaN values in bin_averages and bin_counts
                        valid_avg = ~np.isnan(bin_averages)
                        valid_counts = ~np.isnan(bin_counts)

                        # If any NaNs exist, perform interpolation
                        if np.isnan(bin_averages).any():
                            interp_func_avg = interp1d(bin_centers[valid_avg], bin_averages[valid_avg], kind='linear', bounds_error=False, fill_value="extrapolate")
                            bin_averages = interp_func_avg(bin_centers)

                        if np.isnan(bin_counts).any():
                            interp_func_counts = interp1d(bin_centers[valid_counts], bin_counts[valid_counts], kind='linear', bounds_error=False, fill_value="extrapolate")
                            bin_counts = interp_func_counts(bin_centers)

                        # Create spline fit for smooth curves
                        spline_avg = make_interp_spline(bin_centers, bin_averages, k=3)
                        spline_count = make_interp_spline(bin_centers, bin_counts, k=3)

                        # Generate new x values for a smooth curve
                        xnew = np.linspace(bin_centers.min(), bin_centers.max(), 300)

                        # Compute the smoothed y values from the spline fits
                        y_smooth_avg = spline_avg(xnew)
                        y_smooth_count = spline_count(xnew)

                        # Prepare plot
                        fig = go.Figure()

                        fig.add_trace(go.Scatter(showlegend=False, x=xnew, y=y_smooth_avg, line=dict(color='red'), name='%% of your choice impacting property price'))
                        fig.add_trace(go.Scatter(x=xnew[xnew < 0], y=y_smooth_avg[xnew < 0], fill='tozeroy', line=dict(color='green'), name='Increasing Affordability'))
                        fig.add_trace(go.Scatter(x=xnew[xnew >= 0], y=y_smooth_avg[xnew >= 0], fill='tozeroy', line=dict(color='Red'), name='Increasing Unaffordability'))

                        fig.add_trace(go.Scatter(x=xnew, y=y_smooth_count, name='Sales matching <b>your</b> choice', yaxis='y2', line=dict(color='black', dash='dot')))

                        # Add median line
                        total_area = simps(y_smooth_count, xnew)
                        cumulative_area = np.array([simps(y_smooth_count[:i+1], xnew[:i+1]) for i in range(len(xnew))])
                        half_total_area = total_area / 2
                        index_of_median = np.where(cumulative_area >= half_total_area)[0][0]  # Get the first index where the condition is met
                        median_unaffordability_index = xnew[index_of_median]
                        fig.add_trace(go.Scatter(showlegend=False, x=[median_unaffordability_index, median_unaffordability_index], 
                            y=[0, max(y_smooth_avg.max(), y_smooth_count.max())], mode='lines', 
                            line=dict(color='red', dash='dash'), name='Median Unaffordability'))
                        # Add annotation for median line
                        fig.add_annotation(x=median_unaffordability_index, y=0.9 * y_smooth_avg.max(),  # Adjust y position for text
                                        text=f"Median = {median_unaffordability_index:.2f}", showarrow=True,
                                        arrowhead=1, ax=60, ay=0) 


                        # IMPACT INDEX MEDIAN LINE
                        impact_median = np.median(y_smooth_count)
                        impact_median_on_y1 = ((y_smooth_avg.max() - y_smooth_avg.min())/(y_smooth_count.max() - y_smooth_count.min())) * impact_median

                        fig.add_trace(go.Scatter(showlegend=False, x=[np.min(xnew), np.max(xnew)], 
                            y=[impact_median_on_y1, impact_median_on_y1], mode='lines', 
                            line=dict(color='red', dash='dash'), name='Median Impact'))
                        # Add annotation for median line
                        fig.add_annotation(x=xnew.min() + 0.2 * (xnew.max() - xnew.min()), y=impact_median_on_y1 + 0.01 * (y_smooth_avg.max() - y_smooth_avg.min()),  # Adjust y position for text
                                        text=f"Median = {impact_median_on_y1:.2f}", showarrow=True,
                                        arrowhead=1, ax=0, ay=-30) 


                        desired_height = 600  # Set your desired height
                        aspect_ratio = (8, 6)  # Set your desired aspect ratio (width:height)

                        # Calculate the corresponding width based on the desired height and aspect ratio
                        desired_width = int(desired_height * (aspect_ratio[0] / aspect_ratio[1]))

                        # Layout adjustments for separate y-axes for counts and percentages
                        fig.update_layout(
                            title=f'<i>Unaffordability Impact Profile ({year_gap} year period)</i>\
                            <br> <b>Choice made: ' + custom_choice_text + '</b>',
                            xaxis_title='The Visual Virtuosos <b>Unaffordability Index</b> over all Greater Boston Census Tracts',
                            # yaxis_title='% of <b>your choice</b> impacting unaffordability',
                            yaxis_title='The Visual Virtuosos <b>Impact Index</b>',
                            yaxis_range=[0, max(y_smooth_avg)],
                            yaxis2=dict(
                                title='Frequency of sales matching <b>your</b> choice',
                                overlaying='y',
                                side='right',
                                range=[0, max(y_smooth_count)]
                            ),
                            legend=dict(
                                orientation="h",
                                x=0,
                                y=1.02,
                                xanchor='left',  # Adjust the horizontal alignment of the legend
                                yanchor='bottom'
                            ),
                            plot_bgcolor='white',
                            # legend_title='Legend',
                            width=desired_width,  # Set the calculated width
                            height=desired_height,  # Set the desired height
                            margin=dict(
                                t=125,  # Adjust top margin to leave space for the title
                                b=50,   # Adjust bottom margin to leave space for the x-axis labels
                                l=50,   # Adjust left margin to leave space for the y-axis labels
                                r=50    # Adjust right margin to leave space for the legend
                            )
                        )
                        # fig.show()

                        plot_json = json.dumps(fig, cls=plotly.utils.PlotlyJSONEncoder)
                        with open(f'investor-profiles/public/flip_post/{inv_type}_{budget}_{style}_{subregion}_{flip_var}_index.json', 'w') as file:
                            file.write(plot_json)
                    except Exception as error:
                        # handle the exception
                        print("An exception occurred:", type(error).__name__, "–", error)        
                        print("###################################### FAILED TO GENERATE ######################################")


Small 250000 Traditional SSC NoFlip
Small 250000 Traditional SSC Flip
Small 250000 Traditional ICC NoFlip
Small 250000 Traditional ICC Flip
Small 250000 Traditional MAGIC NoFlip
Small 250000 Traditional MAGIC Flip
Small 250000 Traditional NSPC NoFlip
Small 250000 Traditional NSPC Flip
Small 250000 Traditional NSTF NoFlip
Small 250000 Traditional NSTF Flip
Small 250000 Traditional MWRC NoFlip
Small 250000 Traditional MWRC Flip
Small 250000 Traditional SWAP NoFlip
Small 250000 Traditional SWAP Flip
Small 250000 Traditional TRIC NoFlip
Small 250000 Traditional TRIC Flip
Small 250000 Modern SSC NoFlip
Small 250000 Modern SSC Flip
Small 250000 Modern ICC NoFlip
Small 250000 Modern ICC Flip
Small 250000 Modern MAGIC NoFlip
Small 250000 Modern MAGIC Flip
Small 250000 Modern NSPC NoFlip
Small 250000 Modern NSPC Flip
Small 250000 Modern NSTF NoFlip
Small 250000 Modern NSTF Flip
Small 250000 Modern MWRC NoFlip
Small 250000 Modern MWRC Flip
Small 250000 Modern SWAP NoFlip
Small 250000 Modern SWAP

In [14]:
import json
import plotly.io
import plotly.graph_objects as go

# Assuming your JSON file is named 'plot_data.json'
with open(f'investor-profiles/public/inv_pre/Institutional_profit.json', 'r') as json_file:
    # Load the JSON data from the file
    plot_data = json.load(json_file)

# Extract data, layout, and frames from the loaded JSON data
plotly_data = plot_data['data']
plotly_layout = plot_data['layout']
plotly_frames = plot_data.get('frames', None)

# Create a Plotly figure object
fig = go.Figure(data=plotly_data, layout=plotly_layout, frames=plotly_frames)

# Now you can view the figure
fig.show()

In [6]:
plot_data.keys()

dict_keys(['data', 'layout', 'frames'])

In [9]:
type(plot_json)

str